<center><img src="https://i.imgur.com/kqXDGNG.png"></center>

<center><h1 style="font-family: times-new-roman">📝 COVID-19 Impact on Digital Learning 📝</h1></center>

<h1 style="font-family: times-new-roman">Foreword</h1>
<h2 style="font-family: times-new-roman">A personal Note</h2>
<p style="font-family: times-new-roman">The image above is a picture taken in my mother's classroom. It is empty, but not only because the children are on their Summer Holiday ... it has been empty, ON and OFF, for a while now. Almost two years to be exact.</p>

<p style="font-family: times-new-roman">Covid-19 striked during March 2020 - as for many other countries did. The rest of the school year was spent at home, in some disorganise. My mom started to come to me with all sort of questions she never had before. How do I create an account on this eLearning platform? Can I borrow your laptop? Andrada - the broadband is dead, I cannot join the class. Can I get a graphic design tablet for my birthday to use it as a blackboard?</p>

<p style="font-family: times-new-roman">I heard stories from the school where a math teacher became SUPER creative, building a desk with a bunch of books as standing for the laptop, a desk lamp and his phone suspended (somehow) so he could explain the equation, see the kids, AND send them materials ... all in the same time.</p>

<p style="font-family: times-new-roman">Then the second school year came. It took 5 weeks for the school to completely shut down for another 5 months. Children were getting tired of starring at a screen for 4 hours and then do homework for another 2 ... on the same screens. When the school reopened in April 2021, they came barring all sorts of new behaviors: <i>timidity, pasivity, lack of self confidence, panicks during tests, emotional fragility, and anger issues</i> we saw only in adults until now.</p>

<div class="row">
    <div class="col-sm-8">
      <center><img src="https://i.imgur.com/jbdxyS7.jpg" width=450></center>
  </div>
  <div class="col-sm-4">
    <center><p style="font-family: times-new-roman">They started trading being outside with staying inside. Stopped interracting. And, inevitably, more and more affirmed <q>it was better at home</q>. How can you blame them? To the left is a picture of what the pupil in the last row sees at the table. Exactly. Nothing.</p></center>
  </div>
</div>

<p></p>

<p style="font-family: times-new-roman">This is only my mother's class. She has 28 kids in total, all aged ~8 years old. This is only one school's story. A school like many others in Romania. But how is it the rest of the picture? How did in fact Covid-19 impact the education, from both pupil and teacher perspective? And, most importantly - how big are the damages and what does it take to <b>go back to normal</b>?</p>

<p style="font-family: times-new-roman">Let's find out.</p>

<h4 style="font-family: times-new-roman">Libraries & Helper Functions Below ⬇️</h4>

In [ ]:
# --- CSS STYLE ---
from IPython.core.display import HTML
def css_styling():
    styles = open("../input/2020-cost-of-living/alerts.css", "r").read()
    return HTML("<style>"+styles+"</style>")
css_styling()

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~
# LIBRARIES & ENVIRONMENT
# ~~~~~~~~~~~~~~~~~~~~~~~
import os
import glob
import wandb
import json
import warnings
import imageio
import datetime
import calendar
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML, Javascript
import IPython.display as py_display

# Environment check
warnings.filterwarnings("ignore")
os.environ["WANDB_SILENT"] = "true"
CONFIG = {'competition': 'C19-learn', '_wandb_kernel': 'aot'}

# Secrets
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb")

# Log in W&B personal API
! wandb login $secret_value_0


# ~~~~~~~~~~~~~~~~~~~~~~~
# ~~ HELPER FUNCTIONS ~~
# ~~~~~~~~~~~~~~~~~~~~~~~
def save_dataset_artifact(run_name, artifact_name, path):
    '''Saves dataset to W&B Artifactory.
    run_name: name of the experiment
    artifact_name: under what name should the dataset be stored
    path: path to the dataset'''
    
    run = wandb.init(project='C19-learn', 
                     name=run_name, 
                     config=CONFIG, anonymous="allow")
    artifact = wandb.Artifact(name=artifact_name, 
                              type='dataset')
    artifact.add_file(path)

    wandb.log_artifact(artifact)
    wandb.finish()
    
    
def create_wandb_plot(x_data=None, y_data=None, x_name=None, y_name=None, 
                      title=None, log=None, plot="line"):
    '''Create and save lineplot/barplot in W&B Environment.
    x_data & y_data: Pandas Series containing x & y data
    x_name & y_name: strings containing axis names
    title: title of the graph
    log: string containing name of log'''
    
    data = [[label, val] for (label, val) in zip(x_data, y_data)]
    table = wandb.Table(data=data, columns = [x_name, y_name])
    
    if plot == "line":
        wandb.log({log : wandb.plot.line(table, x_name, y_name, title=title)})
    elif plot == "bar":
        wandb.log({log : wandb.plot.bar(table, x_name, y_name, title=title)})
    elif plot == "scatter":
        wandb.log({log : wandb.plot.scatter(table, x_name, y_name, title=title)})
        
        
def create_wandb_hist(x_data=None, x_name=None, title=None, log=None):
    '''Create and save histogram in W&B Environment.
    x_data: Pandas Series containing x values
    x_name: strings containing axis name
    title: title of the graph
    log: string containing name of log'''
    
    data = [[x] for x in x_data]
    table = wandb.Table(data=data, columns=[x_name])
    wandb.log({log : wandb.plot.histogram(table, x_name, title=title)})
    
    
def append_engagement_data():
    '''Appends all engagement data into 1 big dataframe.
    district_id feature: to separate between districts
    return: full_df (~ 22mil rows)'''
    
    path = "../input/learnplatform-covid19-impact-on-digital-learning/engagement_data"
    all_paths = glob.glob(f"{path}/*")
    all_dfs = []

    for path in all_paths:
        df = pd.read_csv(path)
        df["district_id"] = path.split("/")[-1].split(".")[0]
        all_dfs.append(df)
        
    full_df = pd.concat(all_dfs, axis=0)
    
    # Add time features

    # Holidays
    full_df["holiday"] = np.where(full_df["time"].isin(["2020-01-30", "2020-01-31"]), 1,
            np.where((full_df["time"]>="2020-04-05") & (full_df["time"]<="2020-04-09"), 1,
            np.where((full_df["time"]>="2020-06-12") & (full_df["time"]<="2020-09-10"), 1,
            np.where((full_df["time"]>="2020-11-25") & (full_df["time"]<="2020-11-29"), 1,
            np.where((full_df["time"]>="2020-12-21") & (full_df["time"]<="2020-12-31"), 1, 0)))))

    # Before/After Outbreak
    full_df["outbreak"] = np.where(full_df["time"]<="2020-03-24", 0, 1)
    
    return full_df


def clean_interval(x):
    '''Cleans features with intervals in districts.csv'''
    if pd.isnull(x):
        low, high = pd.np.nan, pd.np.nan
    else:
        x = x.replace("[", "").replace("]", "")
        x = x.split(",")

        low = float(x[0])
        high = float(x[1])
    return low, high


def clean_districts():
    '''Cleans districts.csv dataset and returns it'''
    
    df = districts = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv")
    mapping = pd.read_csv("../input/usa-states-to-region/states.csv")
    mapping = mapping[["State", "State Code", "Region"]]
    mapping.columns = ["State", "Abbreviation", "Region"]

    # Districts: Remove rows with all missing values
    cols = ['state', 'locale', 'pct_black/hispanic',
            'pct_free/reduced', 'county_connections_ratio', 'pp_total_raw']
    df.dropna(subset=cols, how="all", inplace=True)
    df = df.reset_index(drop=True)

    # Add state abbreviation & region
    df = pd.merge(left=df, right=mapping, 
                  left_on="state", right_on="State", 
                  how="left").drop("State", axis=1)

    # Preprocess features with intervals
    cols = ['pct_black/hispanic', 'pct_free/reduced', 
            'county_connections_ratio', 'pp_total_raw']
    for col in cols:
        df[col] = df[col].apply(lambda x: clean_interval(x))
        
    return df

<h4 style="font-family: times-new-roman">Import Data & Preprocess ⬇️</h4>

In [ ]:
# Import data
df = append_engagement_data()
districts = clean_districts()
prods = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")

<div class="alert success-alert" style="font-family: times-new-roman">
  <h4 style="font-family: times-new-roman"><center>🖱️ Don't forget to <b>hover</b> your mouse over the charts and <b>interact</b> with them! 🖱️</center></h4>
</div>


<h1 style="font-family: times-new-roman">1. General Feeling of the Data</h1>

<p style="font-family: times-new-roman">Before going any further, we need to get some clarity of the data and some banchmarks, so our analysis going further will be as non-biased as possible.<p>


<h1 style="font-family: times-new-roman">1.1 The Basics</h1>

<ul style="font-family: times-new-roman">
  <li>There are a total of <b>233 School Districts</b> available within the data, all around <b>USA</b>. A <a href="https://en.wikipedia.org/wiki/School_district">school district</a> is a special-purpose district that operates local public primary and secondary schools in various nations.</li>
  <li>There are a total of <b>372 distinct Educational Technology Products</b>, such as tools like Canva, educational apps like Duolingo, reading sites like Goodreads, or social pages like Facebook.</li>
  <li>The data was collected between 01.01.2020 (a few months before the pandemic hit) until 31.12.2020. This will give a full year overview of a <quote>before and after</quote> usage.</li>
  <li>Summing up all data collected for all the districts we end up with <b>~ 22.3M datapoints</b>, which in the Data Science World would be called Big Data.</li>
  <li>From the 22M datapoints, around <b>24% have missing values</b> in the <code>engagement_index</code> feature. Nevertheless, the <code>pct_access</code> feature is fully available.</li>
  <li><b>Around half of the pupils have at least one page load</b> on a product and on a given day on average. We will see that this average doesn't actually reflect the reality due to unrealistic spikes within the data.</li>
  <li><b>There are almost 168 page loads per 1k students</b> on average on a given product and on a given day. This means <i>close to no page loads</i> for some of the pupils.</li>
</ul>

In [ ]:
run = wandb.init(project='C19-learn', name='basics', 
                 config=CONFIG, anonymous="allow")

# Save basics information
nas = df["engagement_index"].isna().sum()

basics = {"prods": prods["LP ID"].nunique(),
          "districts": df["district_id"].nunique(),
          "country": "USA",
          "time": [df["time"].min(), df["time"].max()],
          "datapoints": len(df),
          "missing": nas/len(df),
          "student_load": df["pct_access"].mean(),
          "page_load": df["engagement_index"].mean()}
# Save into a .json file
with open('basics.json', 'w') as j: json.dump(basics, j)
    
# Save usa.png img as well
im = imageio.imread('../input/covid19-education-data/usa.png')
Image.fromarray(im).save("usa.png")

# Log in into W&B
wandb.log({"prods": int(prods["LP ID"].nunique()),
           "districts": int(basics["districts"]),
           "country": str("USA"),
           "time": str(basics["time"]),
           "datapoints": int(basics["datapoints"]),
           "missing": float(basics["missing"]),
           "student_load": float(basics["student_load"]),
           "page_load": float(basics["page_load"])})
wandb.finish()

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ~~~~~~~~~~~~~~~~~~~~~   CODE   ~~~~~~~~~~~~~~~~~~~~~
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

htmlt1 = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.andradutza {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
	}
	.svg-container-small {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 70%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisColor path{
	stroke: #ffffff;
	}

	.axisColor text{
	stroke: #ffffff;
	}

	.axisColor line{
	stroke: #ffffff;
	}
</style>

<div class="andradutza" style="background-color:#474747;">
	<center><h1 style="color:white;">What are we looking at?</h1></center>
	<img id="baseimg" src="usa.png" style="display:none" />
    <div class="row">
		<div id="map" class="svg-container-small"></div>
	</div>
</div>
'''

js_t1 = '''
require.config({
    paths: {
        d3: "https://d3js.org/d3.v5.min"
     }
     });

require(["d3"], function(d3) {
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/usa.png.*$/, '');

// Set the SVG area
const MARGIN = { LEFT: 100, RIGHT: 10, TOP: 10, BOTTOM: 100 }
const WIDTH = 950 - MARGIN.LEFT - MARGIN.RIGHT
const HEIGHT = 700 - MARGIN.TOP - MARGIN.BOTTOM

const svg = d3.select("#map").append("svg")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 700")
  .classed("svg-content", true);

const g = svg.append("g")
  .attr("transform", `translate(${MARGIN.LEFT}, ${MARGIN.TOP})`)

// USA Map
g.append("image")
 .attr("xlink:href", baseAssetsUrl+"usa.png")
     .attr("x", "3%")
     .attr("y", "5%")
     .attr("width", 320)
     .attr("height", 200)
     .style("stroke", "#6E99FF")
     .style("stroke-width", "1px")
     .style("opacity", 0.6)

// === Data ===
d3.json(baseAssetsUrl+"basics.json").then(function(data){ 

  // ~~~ USA Area ~~~
  // No. districts
  g.append("text")
  .attr("y", "21%")
	.attr("x", "18%")
	.attr("font-size", "45px")
	.attr("text-anchor", "middle")
  .attr("font-weight", 1000)
  .attr("fill", "white")
  .text(data.districts)
  // Title
  g.append("text")
  .attr("y", "4%")
	.attr("x", "18%")
	.attr("font-size", "20px")
	.attr("text-anchor", "middle")
  .attr("font-weight", 400)
  .attr("fill", "white")
  .text("Number of School Districts")

  // ~~~ Products ~~~
  g.append("text")
	.attr("y", "16%")
	.attr("x", "60%")
	.attr("font-size", "35px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 900)
  .attr("fill", "white")
  .text(d3.format(".4n")(data.prods))
  g.append("text")
    .attr("y", "16%")
    .attr("x", "60%")
    .attr("font-size", "20px")
    .attr("text-anchor", "middle")
    .attr("font-weight", 400)
    .attr("fill", "white")
    .attr("dy", "1.5em")
    .text("Educational Tech Products")


  // ~~~ Time scale ~~~
  // Scale
  var xScale = d3.scaleTime()
    .domain([new Date(data.time[0]), new Date(data.time[1])])
    .range([-1, 600])
  var xAxis = d3.axisBottom(xScale)
    .tickFormat(d3.timeFormat("%d %b"))
  g.append("g")
    .attr("class", "axisColor")
    .attr('transform', 'translate('+(WIDTH/12) +','+(HEIGHT/2.1)+')')
    .call(xAxis)
    .selectAll("text")
    .attr("font-size", "12px")
    .attr("font-weight", 0)

  // Text
  g.append("text")
	.attr("y", "39%")
	.attr("x", "68%")
	.attr("font-size", "25px")
	.attr("opacity", "0.6")
  .attr("text-anchor", "middle")
  .attr("fill", "white")
  .attr("font-weight", 900)
  .text("2020")
  g.append("text")
	.attr("y", "39%")
	.attr("x", "17%")
	.attr("font-size", "14px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 800)
  .attr("fill", "white")
  .text("From " + data.time[0] + " to " + data.time[1])

  // ~~~ About dataset ~~~
  g.append("text")
	.attr("y", "55%")
	.attr("x", "20%")
	.attr("font-size", "35px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 900)
  .attr("fill", "white")
  .text(d3.format(",.3s")(data.datapoints))
  g.append("text")
	.attr("y", "55%")
	.attr("x", "20%")
	.attr("font-size", "20px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 500)
  .attr("fill", "white")
  .attr("dy", "1.5em")
  .text("Engagement Datapoints")

  // ~~~ Missing data ~~~
  var dt = {"Not Missing": 1-data.missing,
            "Missing": data.missing}
  var color = d3.scaleOrdinal(['#45caff', '#ff1b6b']);
  var pie = d3.pie()
    .value(function(d) {return d.value; })
  var data_ready = pie(d3.entries(dt))

  var arcGraph = g.selectAll('whatever')
        .data(data_ready)
        .enter()
        .append('path')
        .attr('d', d3.arc()
          .innerRadius(38)
          .outerRadius(55))
        .attr('transform', 
          'translate('+(WIDTH/1.5) +','+(HEIGHT/1.45)+')')
        .attr('fill', function(d, i) {return color(i);})
        .style("opacity", 0.7)      
            
  // Title
  g.append("text")
	.attr("y", "45%")
	.attr("x", "58%")
	.attr("font-size", "20px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 500)
  .attr("fill", "white")
  .attr("dy", "1.5em")
  .text("Missing Data")
  g.append("text")
	.attr("y", "38.5%")
	.attr("x", "59%")
	.attr("font-size", "22px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 900)
  .attr("fill", "white")
  .attr("dy", "6.5em")
  .text(d3.format(",.1%")	(data.missing))


  // ~~~ Progress Bar ~~~
  // Create bar
  const segmentWidth = 230
  g.append('rect')
  .attr("x", "8%")
  .attr("y", "75%")
  .attr('class', 'bg-rect')
  .attr('rx', 6)
  .attr('ry', 10)
  .attr('fill', '#45caff')
  .attr('height', 20)
  .attr('width', segmentWidth)
  .attr("stroke", "#45caff")
  .style("stroke-width", "1.5px")
  .style("opacity", 0.3)

  var progress = g.append('rect')
  .attr('class', 'progress-rect')
  .attr('fill', '#45caff')
  .attr('height', 20)
  .attr('width', 0)
  .attr('rx', 6)
  .attr('ry', 10)
  .attr("x", "8%")
  .attr("y", "75%")
  repeat()

  // Repeat
  function repeat() {
    progress.attr('width', 0)
    .transition()
    .duration(4000)
    .ease(d3.easeExpInOut)
    .attr('width', function(){
      var perc = data.student_load*100;
      return (perc*segmentWidth)/100;
    })
    .on("end", repeat);
  }
  // Text Bar
  g.append("text")
	.attr("y", "74%")
	.attr("x", "12%")
	.attr("font-size", "15px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 700)
  .attr("fill", "white")
  .text("LOADING...")
  g.append("text")
	.attr("y", "74%")
	.attr("x", "30%")
	.attr("font-size", "15px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 700)
  .attr("fill", "white")
  .text(d3.format(",.1%")(data.student_load))
  // Explanation
  g.append("text")
	.attr("y", "83%")
	.attr("x", "20%")
	.attr("font-size", "15px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 400)
  .attr("fill", "white")
  .text("Mean % of students in all districts that have")
  g.append("text")
	.attr("y", "83%")
	.attr("x", "20%")
	.attr("font-size", "15px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 400)
  .attr("fill", "white")
  .attr("dy", "1.5em")
  .text("at least one page load of a given products in a given day")
  

  // ~~~ Loading Page ~~~
  g.append("text")
	.attr("y", "79%")
	.attr("x", "60%")
	.attr("font-size", "35px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 900)
  .attr("fill", "white")
  .text(d3.format(".4r")(data.page_load))
  g.append("text")
	.attr("y", "79%")
	.attr("x", "60%")
	.attr("font-size", "15px")
  .attr("text-anchor", "middle")
  .attr("dy", "1.5em")
  .attr("font-weight", 500)
  .attr("fill", "white")
  .text("Mean Page Loads")
  g.append("text")
	.attr("y", "79%")
	.attr("x", "60%")
	.attr("font-size", "15px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 500)
  .attr("fill", "white")
  .attr("dy", "3em")
  .text("per 1k Students in all districts")
  g.append("text")
	.attr("y", "79%")
	.attr("x", "60%")
	.attr("font-size", "15px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 500)
  .attr("fill", "white")
  .attr("dy", "4.5em")
  .text("on a given day")
});
});'''


h = display(HTML(htmlt1))
j = py_display.Javascript(js_t1)
py_display.display_javascript(j)

<h1 style="font-family: times-new-roman">1.2 School Districts Benchmark</h1>

<p style="font-family: times-new-roman">As said before, we need to get a feel of how the districts are looking BEFORE jumping into the analysis, to avoid any biases in our interpretation as we go further.<p>
    
<p style="font-family: times-new-roman">As general observations, most data was registered in states like <b>Connecticut, Utah, Massachusetts, Illinois or California</b>. More than 50% of the districts are located in the <b>suburbs</b>.<p>
    
<p style="font-family: times-new-roman">Looking at the <quote>wheel</quote> we can see the top 4 characteristics of each state - and whether or not the majority of the schools in the state:</p> 

<ul style="font-family: times-new-roman">
    <li>have more than 50% of students identified as Black/Hispanic;</li>
    <li>have more than 50% of students eligible for free or reduced price lunch;</li>
    <li>have good connection to internet or not (almost all observations had the same interval, so we won't look at this characteristic going further);</li>
    <li>have more than <span>&#36;</span>1,000 per pupil expenditure (marked with deeper red, less than <span>&#36;</span>1,000 marked with lighter red).</li>
</ul>

<div class="alert simple-alert" style="font-family: times-new-roman">
  <p style="font-family: times-new-roman"><b>📌 Note</b>: We can observe that these characteristics correlate: the states where the percentage of Black/Hispanic kids is grater than 50% are also the states where children get some sort of free or reduced lunch.</p>
</div>

In [ ]:
# Get district frequency per state data
states = districts["state"].value_counts().reset_index()
states.columns = ["State", "NoDistricts"]

# Get district frequency per locale data
locale = districts["locale"].value_counts().reset_index()
locale.columns = ["Locale", "NoDistricts"]

# Get state characteristics in terms of districts
characteristics = districts.copy()
cols = ['pct_black/hispanic', 'pct_free/reduced',
        'county_connections_ratio', 'pp_total_raw']
thresh = [0.5, 0.5, 0.5, 10000]

for col in cols:
    characteristics[col] = characteristics[col].apply(lambda x: (x[0]+x[1])/2)
    
characteristics = characteristics\
                .groupby(["state", "Abbreviation", "Region"])[cols]\
                .mean()\
                .reset_index()

for col, t in zip(cols, thresh):
    characteristics[col] = np.where(pd.isnull(characteristics[col]) == True,
                                    'Unclear', np.where(characteristics[col] >= t, 
                                    'Maximum', 'Partial'))
    
characteristics.columns = ["State", "Abbreviation", "Region",
                           "BlackHispanic", "FreeReducedLunch",
                           "ConnectionRatio", "PerPupilExpenditure"]

# Save data
states.to_csv("states.csv", index=False)
locale.to_csv("locale.csv", index=False)
characteristics.to_csv("characteristics.csv", index=False)

# W&B run + save info
run = wandb.init(project='C19-learn', name='dist_basics', 
                 config=CONFIG, anonymous="allow")

create_wandb_plot(x_data=states["State"], y_data=states["NoDistricts"], 
                  x_name="State", y_name="NoDistricts", 
                  title="District Frequency per States",
                  log="dist_state", plot="bar")

create_wandb_plot(x_data=locale["Locale"], y_data=locale["NoDistricts"], 
                  x_name="Locale", y_name="NoDistricts", 
                  title="District Frequency per Locales", 
                  log="dist_locale", plot="bar")
wandb.finish()

htmlt2 = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.andradutza {
		font-family: "Times New Roman", Times, serif;
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	line#limit {
	stroke: #B4C95A;
	stroke-width: 3;
	stroke-dasharray: 8 5;
	}

	text.divergence {
	font-size: 14px;
	fill: #000000;
	}

	text.value {
	font-size: 20px;
	fill: #000000;
	}

	.slider {
	-webkit-appearance: none; 
	appearance: none;
	width: 10%; 
	height: 15px; 
	background: #e7e7e7; 
	outline: none;
	opacity: 0.7; 
	border-radius: 6px;
	-webkit-transition: .2s; 
	transition: opacity .2s;
	}

	/* Mouse-over effects */
	.slider:hover {
	opacity: 1;
	}

	.slider::-webkit-slider-thumb {
	-webkit-appearance: none; /* Override default look */
	appearance: none;
	width: 15px; /* Set a specific slider handle width */
	height: 15px; /* Slider handle height */
	background: #1ACEE6; /* Green background */
	cursor: round; /* Cursor on hover */
	border-radius: 6px;
	}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 0.8rem;
        z-index: 99999;
	}

</style>

<img id="baseimg" src="usa.png" style="display:none" />
<div class="andradutza">
	<center><h1>Districts Overview</h1></center>
    <div class="row">
		<div class="rangeslider">
		</div>
		<div id="map2" class="svg-container"></div>
	</div>
	<div class="row">
		<input type="range" min="1" max="22" 
		value="10" class="slider" id="myRange"
		onchange="updateTextInput(this.value);">
	</div>
</div>
'''




js_t2 = '''
require.config({
  paths: {
    d3: "https://d3js.org/d3.v5.min",
    d3Array: "https://d3js.org/d3-array.v2.min",
  },
});

require(["d3", "d3Array"], function(d3, d3Array) {

// Inject d3array methods to d3
for (var key in d3Array) {
  d3[key] = d3Array[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/usa.png.*$/, '');

// Set the SVG area
const MARGIN = { LEFT: 100, RIGHT: 10, TOP: 10, BOTTOM: 100 }
const WIDTH = 950 - MARGIN.LEFT - MARGIN.RIGHT
const HEIGHT = 900 - MARGIN.TOP - MARGIN.BOTTOM

const svg = d3.select("#map2").append("svg")
  //.attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true);

const g = svg.append("g")
  .attr("transform", `translate(${MARGIN.LEFT}, ${MARGIN.TOP})`)


// === Bar ===
d3.csv(baseAssetsUrl + "states.csv").then(function(data){ 
  
  data.forEach(d => {
    d.NoDistricts = Number(d.NoDistricts)
  })

  // Gradient
  const defs = svg.append('defs');

  const bgGradient = defs
    .append('linearGradient')
    .attr('id', 'bg-gradient')
    .attr('gradientTransform', 'rotate(90)');
  bgGradient
    .append('stop')
    .attr('stop-color', '#f7b227')
    .attr('offset', '0%');
  bgGradient
    .append('stop')
    .attr('stop-color', '#1be9df')
    .attr('offset', '100%');

  // Title
  var title = g.append("text")
	.attr("y", "3%")
	.attr("x", "27%")
	.attr("font-size", "18px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Number of Districts per State")

  // Y label
  const yLabel = g.append("text")
  .attr("class", "y axis-label")
  .attr("x", -(HEIGHT/7))
  .attr("y", -(30))
  .attr("font-size", "15px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .attr("transform", "rotate(-90)")
  .text("Frequency")

  // Scales
  const x = d3.scaleBand()
    .range([0, WIDTH/1.7])
    .domain(data.map(d => d.State))
    .paddingInner(0.3)
    .paddingOuter(0.2)

  const y = d3.scaleLinear()
    .range([HEIGHT/5, 0])
    .domain([0, d3.max(data, d => d.NoDistricts)])

  const xAxisGroup = g.append("g")
    .attr("class", "x axis")
    .attr("transform", `translate(0, ${HEIGHT/4})`)

  const yAxisGroup = g.append("g")
    .attr("class", "y axis")
    .attr("transform", `translate(0, ${WIDTH/22})`)

  // Axis
  const xAxisCall = d3.axisBottom(x)
  xAxisGroup.call(xAxisCall)
    .selectAll("text")
      .attr("y", "0")
      .attr("x", "-10")
      .attr("text-anchor", "end")
      .attr("transform", "rotate(-50)")

  const yAxisCall = d3.axisLeft(y)
    .ticks(7)
    .tickFormat(d => d)
  yAxisGroup.call(yAxisCall)

  // Bar Chart
  const rects = g.selectAll()
    .data(data)
    .enter().append('g')
    .append("rect")
    .attr("transform", `translate(0, ${WIDTH/22})`)
    .attr("y", d => y(d.NoDistricts))
    .attr("x", d => x(d.State))
    .attr('rx', 5)
    .attr('ry', 4)
    .attr("width", x.bandwidth)
    .attr("height", d => HEIGHT/5 - y(d.NoDistricts))
    .style("fill", "url(#bg-gradient)")
    .style("fill", "url(#bg-gradient)")
    .style("stroke-width", "1px")


  // Interactive
  var div = d3.select("body").append("div")
    .attr("class", "tooltip-donut")
    .style("opacity", 0);

  rects
    .on('mouseenter', function (actual, i) {
    // MOUSE ON
    d3.selectAll('.value')
          .attr('opacity', 0)

    d3.select(this)
        .transition()
        .duration(100)
        .attr('opacity', 0.3)
        .attr('x', (a) => x(a.State) - 2.5)
        .attr('width', x.bandwidth() + 5)

    // Add line
    const y_new = y(actual.NoDistricts)
    var line = g.append('line')
          .attr('id', 'limit')
          .attr('x1', 0)
          .attr('y1', y_new + 35)
          .attr('x2', 500)
          .attr('y2', y_new + 35)
          .attr('stroke', '#B4C95A')

    // Makes the new div appear
    div.transition()
        .duration(50)
        .style("opacity", 1);
    let num = data[i].NoDistricts;
    div.html(num)
        .style("left", (d3.event.pageX) + "px")
        .style("top", (d3.event.pageY-30) + "px");


  })
    // MOUSE LEAVE
    .on('mouseleave', function () {
      d3.selectAll('.value')
          .attr('opacity', 1)

      d3.select(this)
      .transition()
      .duration(100)
      .attr('opacity', 1)
      .attr('x', (a) => x(a.State))
      .attr('width', x.bandwidth())

      // New div dissapears
      div.transition()
          .duration('50')
          .style("opacity", 0);

      g.selectAll('#limit').remove()
      g.selectAll('.divergence').remove()
    })

});

// === Pie ===
d3.csv(baseAssetsUrl + "locale.csv").then(function(data){ 

  data.forEach(d => {
    d.NoDistricts = Number(d.NoDistricts)
  })

  // Title
  var title = g.append("text")
    .attr("y", "3%")
    .attr("x", "70%")
    .attr("font-size", "18px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Locales Frequency")

  var color = d3.scaleOrdinal(['#1BE9DF', '#B4C95A',
                                 '#F7B227', '#E16853'])

  var dt = {"Suburb":104, "Rural":33,
            "City":29, "Town":10}

  var arc = d3.arc()
    .innerRadius(55)
    .outerRadius(89);

  var pie = d3.pie()
    .value(function (d) {
        return d.value;
    })

  var div = d3.select("body").append("div")
    .attr("class", "tooltip-donut")
    .style("opacity", 0);

  // Pie
  var path = g.selectAll('smth')
  .data(pie(d3.entries(dt)))
  .enter()
  .append('path')
  .attr('d', arc)
  .attr('fill', function(d, i) {return color(i);})
  .attr("stroke", "white")
    .style("stroke-width", "1px")
  .attr('transform', 
   'translate('+(WIDTH/1.25) +','+(HEIGHT/6.5)+')')
   // INTERACTIVE
  .on('mouseover', function (d, i) {
      d3.select(this).transition()
          .duration('50')
          .attr('opacity', '.5');
      // Makes the new div appear
      div.transition()
          .duration(50)
          .style("opacity", 1);
      let num = data[i].NoDistricts;
      div.html(num)
          .style("left", (d3.event.pageX + 10) + "px")
          .style("top", (d3.event.pageY - 15) + "px");

  })
  .on('mouseout', function (d, i) {
      d3.select(this).transition()
          .duration('50')
          .attr('opacity', '1');
      // New div dissapears
      div.transition()
          .duration('50')
          .style("opacity", 0);
  });
  

  // Legend
  var legendG = g.selectAll(".legend")
    .data(pie(data))
    .enter().append("g")
    .attr("transform", function(d,i){
      return 'translate(' + (WIDTH - 190) + "," + (i * 15+95) + ')'
    })
    .attr("class", "legend");   

  legendG.append("rect")
    .attr("width", 12)
    .attr("height", 12)
    .attr("fill", function(d, i) {
      return color(i);
    })
    .attr("stroke", "#white")
    .style("stroke-width", "1px");

  legendG.append("text")
    .text(function(d) {return d.data.Locale;})
    .style("font-size", 12)
    .attr("y", 10)
    .attr("x", 13);

  
});

// === Wheel ===
d3.csv(baseAssetsUrl + "characteristics.csv").then(function(data){

  // Wheel Rotation
  var rotate = document.getElementById("myRange").value;

  // Base Data
  const regionOrder = ["Northeast", "Midwest", "West", "South"]
  var wideData = data.sort(
    (a, b) =>
      d3.ascending(
        regionOrder.indexOf(a.Region),
        regionOrder.indexOf(b.Region)
      ) || d3.ascending(a.Abbreviation, b.Abbreviation)
  )

  // === Constants ===
  var rotation = (rotate / wideData.length) * Math.PI * 2
  var categories = wideData.columns.slice(3)
  var categoryOrder = categories.slice().reverse()
  const innerRadius = 135
  const outerRadius = WIDTH / 3
  const colors = ["#1acee6", "#5d6dba", "#af6eac", "#c1385c"]

  var grouped = d3.groups(wideData, d => d.Region)
  var color = d3
    .scaleOrdinal()
    .domain(categories)
    .range(colors)
    .unknown("rgb(234,234,234)")

  // Scales
  const x = d3
    .scaleBand()
    .domain(wideData.map(d => d.State))
    .range([rotation, 2 * Math.PI + rotation])

  const y = d3.scaleLinear()
    .domain([0, categories.length])
    .range([innerRadius, outerRadius])

  // Regions Data
  var regions = d3
    .groups(wideData, d => d.Region)
    .map(([region, states]) => {
      const startAngle = x(states[0].State);
      const endAngle = 
          x(states[states.length - 1].State) + x.bandwidth();
      const labelAngle =
        ((startAngle + endAngle) / 2 - Math.PI / 2) % (Math.PI * 2);
      return {
        region,
        startAngle,
        endAngle,
        labelAngle
      };
    })

  // Long Data (for main chart)
  var longData = wideData.flatMap(row =>
    categories.map(category => ({
      state: row.State,
      abbreviation: row.Abbreviation,
      region: row.Region,
      category,
      status: row[category]
    }))
  )

  // ARC
  var arc = d3
  .arc()
  .innerRadius(d => y(categoryOrder.indexOf(d.category)))
  .outerRadius(d => y(categoryOrder.indexOf(d.category) + 1))
  .startAngle(d => x(d.state))
  .endAngle(d => x(d.state) + x.bandwidth())
  .padRadius(innerRadius)

  // Label X and Y
  var labelX = (angle, radius) => Math.cos(angle) * radius
  var labelY = (angle, radius) => Math.sin(angle) * radius

  // Banned X and Y
  var bannedX = d =>
    labelX(
      x(d.state) + x.bandwidth() / 2 - Math.PI / 2,
      y(categoryOrder.indexOf(d.category) + 0.5)
    )
  var bannedY = d =>
    labelY(
      x(d.state) + x.bandwidth() / 2 - Math.PI / 2,
      y(categoryOrder.indexOf(d.category) + 0.5)
    )


  // ====================================
  // =============== CHART ==============
  // ====================================
  const wheel = g.selectAll()
  .data(longData)
  .join("g");

  wheel.append("path")
    .attr("d", arc)
    .attr("stroke", "white")
    .attr("fill", d =>
      color(["Maximum", "Partial"]
      .includes(d.status) ? d.category : null))
    .attr("fill-opacity", 
      d => (d.status === "Partial" ? 0.4 : 1))
    .attr('transform', 
    'translate('+(WIDTH/2.2) +','+(HEIGHT/1.35)+')');

  wheel
    .append("title")
    .text(d => `${d.state}: ${d.category}, ${d.status}`);

  // Region Labels
  var labelArc = d3
  .arc()
  .innerRadius(innerRadius - 20)
  .outerRadius(innerRadius - 20)
  .startAngle(d => d.startAngle)
  .endAngle(d => d.endAngle)

  // State Labels
  wheel
    .append("g")
    .attr("text-anchor", "middle")
    .attr("fill", "#888")
    .selectAll("text")
    .data(wideData)
    .join("text")
    .attr("x", d =>
      labelX(x(d.State) + x.bandwidth() / 2 - Math.PI / 2, outerRadius + 16)
    )
    .attr("y", d =>
      labelY(x(d.State) + x.bandwidth() / 2 - Math.PI / 2, outerRadius + 16)
    )
    .attr("dy", "0.31em")
    .text(d => d.Abbreviation)
    .attr('transform', 
    'translate('+(WIDTH/2.2) +','+(HEIGHT/1.35)+')');

  // LEGEND
  const legend = g
    .selectAll("wheel")
    .data(categories.concat(["Unclear"]))
    .join("g")
    .attr(
      "transform",
      (d, i) => `translate(325,
          ${(i - ((categories.length + 1)/5)+28) * 20})`
    );

  legend
    .filter(d => categories.includes(d))
    .append("rect")
    .attr("x", -50)
    .attr("width", 36)
    .attr("height", 18)
    .style("opacity", 0.4)
    .attr("fill", color);

  legend
    .append("rect")
    .attr("x", 0)
    .attr("width", 30)
    .attr("height", 18)
    .attr("fill", color);

  legend
    .append("text")
    .attr("x", 42)
    .attr("y", 9)
    .attr("dy", "0.35em")
    .attr("font-size", "12px")
    .text(d => d);

  g.append("text")
    .attr("y", "59.4%")
    .attr("x", "31%")
    .attr("font-size", "12px")
    .attr("text-anchor", "middle")
    .attr("font-weight", 100)
    .text("<50%")

  g.append("text")
    .attr("y", "59.4%")
    .attr("x", "36%")
    .attr("font-size", "12px")
    .attr("text-anchor", "middle")
    .attr("font-weight", 100)
    .text(">50%")

  // Title
  g.append("text")
    .attr("y", "37%")
    .attr("x", "75%")
    .attr("font-size", "20px")
    .attr("text-anchor", "middle")
    .attr("font-weight", 600)
    .text("State")
  g.append("text")
    .attr("y", "37%")
    .attr("x", "75%")
    .attr("dy", "1.5em")
    .attr("font-size", "20px")
    .attr("text-anchor", "middle")
    .attr("font-weight", 600)
    .text("Characteristics")
});

});'''


h = display(HTML(htmlt2))
j = py_display.Javascript(js_t2)
py_display.display_javascript(j)

<h1 style="font-family: times-new-roman">2. The State of Digital Learning</h1>

<h2 style="font-family: times-new-roman">2.1 Timeline for School Year 2020</h2>

<div class="row">
  <div class="col-sm-8">
    <center><img src="https://i.imgur.com/qebeciu.png" width=500></center>
  </div>
    
  <div class="col-sm-4">
    <p style="font-family: times-new-roman">After <a href="https://www.ajmc.com/view/what-were-reading-roots-of-chinese-illness-discovered-birth-costs-soar-public-health-emergency-in-puerto-rico">WHO Announced Mysterious Coronavirus Pneumonia in Wuhan in January 2020</a>, the apparition of the <a href="https://www.cdc.gov/media/releases/2020/p0121-novel-coronavirus-travel-case.html">first COVID-19 case</a> on 21st January and the USA declaring the new coronavirus a Health Emergency (and then a Pandemic), schools started to cieze all on site activity <a href="https://www.edweek.org/leadership/map-coronavirus-and-school-closures-in-2019-2020/2020/03">between 16th and 24th of March</a> 2020.<p>
    <p style="font-family: times-new-roman">Depending on the state and district, they <a href="https://www.thecut.com/2020/09/will-schools-open-in-the-fall-reopening-statuses-explained.html">resumed activity</a> at the beginning of the next school year (2020-2021) either full time, hybrid/part-time or still fully remote. Connecticut decided to open more than 50% of it's schools, but <a href="https://gothamist.com/news/coronavirus-updates-trump-violates-public-health-guidelines-hold-indoor-rally-nevada?mc_cid=f37a7512a3&mc_eid=1b2da7088e">shut down 12 of them</a> in the first weeks due to positive COVID-19 testing of both staff and pupils.<p>
  </div>
</div>

<h3 style="font-family: times-new-roman">📅 Student Activity Through Time</h3>

<p style="font-family: times-new-roman">A big graph with lots of information; Ok, so what are the key takeaways?<p>
    
<ul style="font-family: times-new-roman">
  <li>For the purpose of readability, this graph was created using only top <b>10 states</b> with most frequent districts (hence available data).</li>
  <li>The median percentage of active users on a given day for a product varies widely, from <b>New York having on average ~0.14% activity</b> to states like <b>California or Washington with only 0.1% active users per product</b> on a given day.</li>
  <li>The Activity Trend is general for almost all states - with <b>increased activity during the school year</b> and <b>dormant activity during holidays</b>, like the Summer Break or towards the end of the year (Winter Recess).</li>
</ul>

<div class="alert simple-alert" style="font-family: times-new-roman">
  <p style="font-family: times-new-roman"><b>📌 Note</b>: Observe that, although the total <b>mean</b> percentage of students in the district that have registered activity for 1 product is <b>~ 50%</b>, the <b>median falls at only 0.2%</b>. This means that the data is extremely squewed for some of the products (we'll discover more in the next chapters).</p>
</div>

In [ ]:
timeline = df[["time", "pct_access", "district_id"]]

# Make district_id string
districts["district_id"] = districts["district_id"].astype(str)

# *Some rows are missing
state_filter = ["CT", "UT", "MA", "IL", "CA", "OH", "NY", "IN", "WA", "MO"]
timeline = pd.merge(left=timeline, right=districts,
                    on="district_id", how="inner")
timeline = timeline.groupby(["time", "Abbreviation"])["pct_access"]\
                    .median().reset_index()
timeline["pct_access"] = timeline["pct_access"] * 100

# Filter top 10 states
timeline = timeline[timeline["Abbreviation"].isin(state_filter)]

# Pivot states on columns
timeline = pd.pivot(timeline, index="time",
                    columns="Abbreviation", values="pct_access")\
                    .reset_index()
timeline = timeline.fillna(0)

# Convert from string to number
string_date = timeline["time"]
timeline.insert(0, 'time2', string_date)
timeline["time"] = pd.to_datetime(timeline["time"]).apply(lambda x: x.value)
for col in timeline.columns[1:]: 
    timeline[col] = timeline[col].astype("int64")

# Save to .csv
timeline.to_csv("timeline.csv", index=False)

# Save usa.png img as well
im = imageio.imread('../input/covid19-education-data/schoolbreak.png')
Image.fromarray(im).save("schoolbreak.png")


htmlt3 = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.andradutza {
		font-family: "Times New Roman", Times, serif;
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}
	.area-label {
        font-family: sans-serif;
        fill-opacity: 1;
        fill: white;
      }

</style>

<img id="baseimg" src="usa.png" style="display:none" />
<div class="andradutza">
	<center><h1>School Timeline 2020</h1></center>
	<center><h4>-Median Daily Student Activity per Product-</h4></center>
    <div class="row">
		<div id="map3" class="svg-container"></div>
	</div>
</div>
'''


js_t3 = '''
require.config({
  paths: {
    d3v6: "https://d3js.org/d3.v6.min",
    moment: "https://unpkg.com/moment@2.29.1/moment",
  }
});

require(["d3v6", "moment"], function(d3, moment) {

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/usa.png.*$/, '');

// Set the SVG area
const margin = { LEFT: 100, RIGHT: 10, TOP: 10, BOTTOM: 100 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM
const chart_size = 35

const svg = d3.select("#map3").append("svg")
  // .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)


svg.append("image")
.attr("xlink:href", baseAssetsUrl + "schoolbreak.png")
    .attr("x", "-1%")
    .attr("y", "41%")
    .attr("width", 280)
    .attr("height", 200)
    .style("opacity", 1);


d3.csv(baseAssetsUrl + "timeline.csv").then((data) => {
  
  // List of groups = header of the csv files
  var keys = data.columns.slice(2)
  var days = ["Mar 16","Mar 24","Jun 12","Sep 10"];
  var formatDay = (d, i) => {
      return days[i];      
  }

  // Stacking
  var series = d3.stack()
    .keys(keys)
    .offset(d3.stackOffsetSilhouette)
    (data)

  // Area Chart
  var area = d3.area()
  .y(d => y(d.data.time))
  .x0(d => x(d[0]))
  .x1(d => x(d[1]))

  // X and Y axis
  var y = d3.scaleLinear()
    .domain(d3.extent(data, d => d.time))
    .range([0, width])

  var x = d3.scaleLinear()
  .domain([-chart_size, chart_size])
  .range([height, 0])

  // Color
  const color = d3.scaleOrdinal()
    .domain(keys)
    .range(["#db222a", "#e16853", "#f7b227", "#b4c95a", 
              "#1be9df", "#1acee6", "#5d6dba", 
              "#5c4d86", "#af6eac", "#c1385c"])

  // X axis placement
  var yAxis = g => g
    .attr("transform", "translate(0," + (height/2-1) + ")")
    .call(d3.axisLeft(y)
            .tickSize(-height+100)
            .tickValues([1584316800000000000,1585008000000000000,
              1591920000000000000, 1599696000000000000])
            .tickFormat(formatDay)
            )
    .call(g => g.select(".domain").remove())
    .call(g => g.selectAll(".tick line")
                .attr("stroke", "#989898")
                .attr('stroke-dasharray', '5 5'))
    .call(g => g.selectAll(".tick")
                .select('text')
                .attr('fill', "#989898")
                .style("font-size", 13))
                .attr("transform", "translate(50,0)")

  // Show
  const path = svg.append("g")
    .selectAll("path")
    .data(series)
    .join("path")
      .attr("data-genre", d => d.key)
      .attr("fill", ({key}) => color(key))
      .attr("d", area)
      .attr("opacity", 0.8)

  svg.append("g")
      .call(yAxis)

  // ======================================
  const hover = (svg, path) => {
  
    const line = svg.append("g")
         .attr("display", "none");
  
    line.append("g")
        .selectAll("line")
        .data(series)
        .join("line")
        .attr("class", "cursor-line")
        .attr("fill", "#fff")
        .attr("stroke-width", 0.8)
        .attr("y1", 10)
        .attr("x1", width-50)
        .attr("y2", 10)
        .attr("x2", 10);
  
    line.append("text")
        .attr("class", "text-year")
        .attr("font-size", 13)
        .attr("x", 0)
        .attr("y", 26)
        .attr("transform", 
        "rotate(0 0 0) translate(10, -20)");
    
    line.append("g")
        .selectAll("text")
        .data(series)
        .join("text")
        .attr("class", "text-genre")
        .attr("font-weight", "bold")
        .attr("font-size", 13)
        .attr("x", width/2+350)
        .attr("y", 26)
        .attr("transform", 
        "rotate(0 0 10)");

    line.append("g")
        .selectAll("text")
        .data(data)
        .join("text")
        .attr("class", "text-rating")
        .attr("font-size", 13)
        .attr("x", width/2+335)
        .attr("y", 1);

    const mousemove = (event) => {
  
      event?.preventDefault();
      // pointer = [posX, posY]
      const pointer = d3.pointer(event);
          
      const xm = x.invert(pointer[0]);
      const ym = y.invert(pointer[1]);
      const genre = d3.select(event.target).attr("data-genre");
  
      line.attr("transform", `translate(0 ${pointer[1]})`)
          .style("visibility", "inherit");
      
      const dateFormatted = moment.unix(parseInt(ym)/1e9).format("MMM Do");
      line.select(".text-year")
        .text(dateFormatted);
      
      line.selectAll(".text-rating")
        .text(d => {
          return Math.floor(parseInt(d.time)/(24*60*60*1e9)) === Math.floor(parseInt(ym)/(24*60*60*1e9)) ? (d[genre]/100)+"%" : "";
        });

      line.selectAll(".text-genre")
        .style("visibility", "hidden")
        .filter(d => d.key === genre)
        .style("visibility", "inherit")
        .attr("fill", d => d.key === genre ? color(d.key) : "#000")
        .text(genre);
      
      line.selectAll(".cursor-line")
        .attr("stroke", "fff0")
        .filter(d => d.key === genre)
        .attr("stroke", d => d.key === genre ? color(d.key) : "#fff0")
      
      path.attr("opacity", d => d.key === genre ? 0.8 : 0.3);
      
      if (genre === null) {
        path.attr("opacity", 0.8);
        line.style("visibility", "hidden");
      }
    }

    const mouseenter = () => {
      line.attr("display", null);
    }
  
    const mouseleave = () => {
      line.attr("display", "none");
    }

    svg
        .on("mousemove", mousemove)
        .on("mouseenter", mouseenter)
        .on("mouseleave", mouseleave)

  }

  svg.call(hover, path);

  // Pandemic Start
  svg.append("text")
  .attr("x", "42%")
  .attr("y", "21%")
  .attr("font-size", "20px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 900)
  .style("pointer-events", "none")
  .text("Pandemic Start");

  // Increase
  svg.append("text")
  .attr("x", "10%")
  .attr("y", "30%")
  .attr("font-size", "13px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 900)
  .text("Increase for all states");

  svg.append("text")
  .attr("x", "10%")
  .attr("y", "30%")
  .attr("dy", "1.5em")
  .attr("font-size", "10px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 100)
  .text("There can be seen an increase");
  svg.append("text")
  .attr("x", "10%")
  .attr("y", "30%")
  .attr("dy", "2.5em")
  .attr("font-size", "10px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 100)
  .text("in activity for all states");
  svg.append("text")
  .attr("x", "10%")
  .attr("y", "30%")
  .attr("dy", "3.5em")
  .attr("font-size", "10px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 100)
  .text("after the official start of the pandemic.");

  // Arrow
  svg.append("svg:defs").append("svg:marker")
  .attr("id", "triangle")
  .attr("refX", 6)
  .attr("refY", 6)
  .attr("markerWidth", 30)
  .attr("markerHeight", 30)
  .attr("markerUnits","userSpaceOnUse")
  .attr("orient", "auto")
  .append("path")
  .attr("d", "M 0 0 12 6 0 12 3 6")
  .style("fill", "#989898");

  svg.append("path")
    .attr('d', d3.line()([[40, 250], [150, 250]]))
    .attr("stroke-width", 2)
    .attr("stroke", "#989898")
    .attr("marker-end", "url(#triangle)");

  // Decrease
  svg.append("text")
  .attr("x", "74%")
  .attr("y", "76%")
  .attr("font-size", "13px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 900)
  .text("Decrease in activity");

  svg.append("text")
  .attr("x", "74%")
  .attr("y", "76%")
  .attr("dy", "1.5em")
  .attr("font-size", "10px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 100)
  .text("There can be seen a decrease in activity");
  svg.append("text")
  .attr("x", "74%")
  .attr("y", "76%")
  .attr("dy", "2.5em")
  .attr("font-size", "10px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 100)
  .text("for the new academic year 2020-2021.");

  svg.append("path")
    .attr('d', d3.line()([[790, 720], [620, 720]]))
    .attr("stroke-width", 2)
    .attr("stroke", "#989898")
    .attr("marker-end", "url(#triangle)");

});

});
'''

h = display(HTML(htmlt3))
j = py_display.Javascript(js_t3)
py_display.display_javascript(j)

<h3 style="font-family: times-new-roman">📚 Student Engagement Through Time</h3>

<p style="font-family: times-new-roman">The engagement's pattern looks very similar with the activity one:<p>
    
<ul style="font-family: times-new-roman">
  <li>Again, as a diclaimer, this graph was created using only top <b>10 states</b> with most frequent districts (hence available data).</li>
  <li>After the start of the pandemic and the official closing of schools for the remainer of the year, the <b>median page loads per product has increased visibly</b> for most of the states.</li>
   <li>Compared to the activity analyzed above, the <b>engagement doesn't decrease so drastically during the summer holiday</b>. This is because lots of Educational Products are used in the free time too.</li>
  <li>Although the second part of the year (when the School Year 2020-2021 started) most of the states opted for hybrid or from home study, there is a <b>slight decrease in the median engagement</b> per product per 1k students.</li>
</ul>

<div class="alert simple-alert" style="font-family: times-new-roman">
  <p style="font-family: times-new-roman"><b>📌 Note</b>: The two streamgraphs show the same possibly alarming pattern. <i>Are they getting bored</i>? Why is there a decrease in the new school year in activity and engagement on these products?</p>
</div>

In [ ]:
timeline2 = df[["time", "engagement_index", "district_id"]]

# *Some rows are missing
state_filter = ["CT", "UT", "MA", "IL", "CA", "OH", "NY", "IN", "WA", "MO"]
timeline2 = pd.merge(left=timeline2, right=districts,
                    on="district_id", how="inner")
timeline2 = timeline2.groupby(["time", "Abbreviation"])["engagement_index"]\
                    .median().reset_index()

# Filter top 10 states
timeline2 = timeline2[timeline2["Abbreviation"].isin(state_filter)]

# Pivot states on columns
timeline2 = pd.pivot(timeline2, index="time",
                    columns="Abbreviation", values="engagement_index")\
                    .reset_index()
timeline2 = timeline2.fillna(0)

# Convert from string to number
string_date = timeline2["time"]
timeline2.insert(0, 'time2', string_date)
timeline2["time"] = pd.to_datetime(timeline2["time"]).apply(lambda x: x.value)
for col in timeline2.columns[1:]: 
    timeline2[col] = timeline2[col].astype("int64")

# Save to .csv
timeline2.to_csv("timeline2.csv", index=False)


htmlt4 = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.andradutza {
		font-family: "Times New Roman", Times, serif;
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}
	.area-label {
        font-family: sans-serif;
        fill-opacity: 1;
        fill: white;
      }
</style>

<img id="baseimg" src="usa.png" style="display:none" />
<div class="andradutza">
	<center><h1>School Timeline 2020</h1></center>
	<center><h4>-Median Daily Student Engagement per Product-</h4></center>
    <div class="row">
		<div id="map31" class="svg-container"></div>
	</div>
</div>
'''


js_t4 = '''
require.config({
  paths: {
    d3v6: "https://d3js.org/d3.v6.min",
    moment: "https://unpkg.com/moment@2.29.1/moment",
  }
});

require(["d3v6", "moment"], function(d3, moment) {

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/usa.png.*$/, '');

// Set the SVG area
const margin = { LEFT: 100, RIGHT: 10, TOP: 10, BOTTOM: 100 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM
const chart_size = 35

const svg = d3.select("#map31").append("svg")
  // .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)


svg.append("image")
.attr("xlink:href", baseAssetsUrl + "schoolbreak.png")
    .attr("x", "-1%")
    .attr("y", "41%")
    .attr("width", 280)
    .attr("height", 200)
    .style("opacity", 1);


d3.csv(baseAssetsUrl + "timeline2.csv").then((data) => {
  
  // List of groups = header of the csv files
  var keys = data.columns.slice(2)
  var days = ["Mar 16","Mar 24","Jun 12","Sep 10"];
  var formatDay = (d, i) => {
      return days[i];      
  }

  // Stacking
  var series = d3.stack()
    .keys(keys)
    .offset(d3.stackOffsetSilhouette)
    (data)

  // Area Chart
  var area = d3.area()
  .y(d => y(d.data.time))
  .x0(d => x(d[0]))
  .x1(d => x(d[1]))

  // X and Y axis
  var y = d3.scaleLinear()
    .domain(d3.extent(data, d => d.time))
    .range([0, width])

  var x = d3.scaleLinear()
  .domain([-chart_size, chart_size])
  .range([height, 0])

  // Color
  const color = d3.scaleOrdinal()
    .domain(keys)
    .range(["#db222a", "#e16853", "#f7b227", "#b4c95a", 
              "#1be9df", "#1acee6", "#5d6dba", 
              "#5c4d86", "#af6eac", "#c1385c"])

  // X axis placement
  var yAxis = g => g
    .attr("transform", "translate(0," + (height/2-1) + ")")
    .call(d3.axisLeft(y)
            .tickSize(-height+100)
            .tickValues([1584316800000000000,1585008000000000000,
              1591920000000000000, 1599696000000000000])
            .tickFormat(formatDay)
            )
    .call(g => g.select(".domain").remove())
    .call(g => g.selectAll(".tick line")
                .attr("stroke", "#989898")
                .attr('stroke-dasharray', '5 5'))
    .call(g => g.selectAll(".tick")
                .select('text')
                .attr('fill', "#989898")
                .style("font-size", 13))
                .attr("transform", "translate(50,0)")

  // Show
  const path = svg.append("g")
    .selectAll("path")
    .data(series)
    .join("path")
      .attr("data-genre", d => d.key)
      .attr("fill", ({key}) => color(key))
      .attr("d", area)
      .attr("opacity", 0.8)

  svg.append("g")
      .call(yAxis)

  // ======================================
  const hover = (svg, path) => {
  
    const line = svg.append("g")
         .attr("display", "none");
  
    line.append("g")
        .selectAll("line")
        .data(series)
        .join("line")
        .attr("class", "cursor-line")
        .attr("fill", "#fff")
        .attr("stroke-width", 0.8)
        .attr("y1", 10)
        .attr("x1", width-50)
        .attr("y2", 10)
        .attr("x2", 10);
  
    line.append("text")
        .attr("class", "text-year")
        .attr("font-size", 13)
        .attr("x", 0)
        .attr("y", 26)
        .attr("transform", 
        "rotate(0 0 0) translate(10, -20)");
    
    line.append("g")
        .selectAll("text")
        .data(series)
        .join("text")
        .attr("class", "text-genre")
        .attr("font-weight", "bold")
        .attr("font-size", 13)
        .attr("x", width/2+350)
        .attr("y", 26)
        .attr("transform", 
        "rotate(0 0 10)");

    line.append("g")
        .selectAll("text")
        .data(data)
        .join("text")
        .attr("class", "text-rating")
        .attr("font-size", 13)
        .attr("x", width/2+237)
        .attr("y", 1);

    const mousemove = (event) => {
  
      event?.preventDefault();
      // pointer = [posX, posY]
      const pointer = d3.pointer(event);
          
      const xm = x.invert(pointer[0]);
      const ym = y.invert(pointer[1]);
      const genre = d3.select(event.target).attr("data-genre");
  
      line.attr("transform", `translate(0 ${pointer[1]})`)
          .style("visibility", "inherit");
      
      const dateFormatted = moment.unix(parseInt(ym)/1e9).format("MMM Do");
      line.select(".text-year")
        .text(dateFormatted);
      
      line.selectAll(".text-rating")
        .text(d => {
          return Math.floor(parseInt(d.time)/(24*60*60*1e9)) === Math.floor(parseInt(ym)/(24*60*60*1e9)) ? d[genre]+" page loads/ 1k students" : "";
        });

      line.selectAll(".text-genre")
        .style("visibility", "hidden")
        .filter(d => d.key === genre)
        .style("visibility", "inherit")
        .attr("fill", d => d.key === genre ? color(d.key) : "#000")
        .text(genre);
      
      line.selectAll(".cursor-line")
        .attr("stroke", "fff0")
        .filter(d => d.key === genre)
        .attr("stroke", d => d.key === genre ? color(d.key) : "#fff0")
      
      path.attr("opacity", d => d.key === genre ? 0.8 : 0.3);
      
      if (genre === null) {
        path.attr("opacity", 0.8);
        line.style("visibility", "hidden");
      }
    }

    const mouseenter = () => {
      line.attr("display", null);
    }
  
    const mouseleave = () => {
      line.attr("display", "none");
    }

    svg
        .on("mousemove", mousemove)
        .on("mouseenter", mouseenter)
        .on("mouseleave", mouseleave)

  }

  svg.call(hover, path);

  // Pandemic Start
  svg.append("text")
  .attr("x", "42%")
  .attr("y", "21%")
  .attr("font-size", "20px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 900)
  .style("pointer-events", "none")
  .text("Pandemic Start");

  // Annomaly 11 April NY
  svg.append("text")
  .attr("x", "10%")
  .attr("y", "32%")
  .attr("font-size", "13px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 900)
  .text("New York Spike in Ed. Products");
  svg.append("text")
  .attr("x", "10%")
  .attr("y", "32%")
  .attr("dy", "1em")
  .attr("font-size", "12px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 900)
  .text("11th April");

  svg.append("text")
  .attr("x", "10%")
  .attr("y", "32%")
  .attr("dy", "2.5em")
  .attr("font-size", "10px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 100)
  .text("Most proeminent products were");
  svg.append("text")
  .attr("x", "10%")
  .attr("y", "32%")
  .attr("dy", "3.5em")
  .attr("font-size", "10px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 100)
  .text("Google Docs/Drive/Forms/Sheets/Classroom");
  svg.append("text")
  .attr("x", "10%")
  .attr("y", "32%")
  .attr("dy", "4.5em")
  .attr("font-size", "10px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 100)
  .text("Duolingo, CoolMath Games & SoundCloud");

  // Arrow
  svg.append("svg:defs").append("svg:marker")
  .attr("id", "triangle")
  .attr("refX", 6)
  .attr("refY", 6)
  .attr("markerWidth", 30)
  .attr("markerHeight", 30)
  .attr("markerUnits","userSpaceOnUse")
  .attr("orient", "auto")
  .append("path")
  .attr("d", "M 0 0 12 6 0 12 3 6")
  .style("fill", "#989898");

  svg.append("path")
    .attr('d', d3.line()([[100, 275], [100, 240]]))
    .attr("stroke-width", 2)
    .attr("stroke", "#989898")
    .attr("marker-end", "url(#triangle)");


  // Annomaly 11 April NY
  svg.append("text")
  .attr("x", "74%")
  .attr("y", "76%")
  .attr("font-size", "13px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 900)
  .text("Slight overall decrease");

  svg.append("text")
  .attr("x", "74%")
  .attr("y", "76%")
  .attr("dy", "1.5em")
  .attr("font-size", "10px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 100)
  .text("There is a decrease in activity for all states");
  svg.append("text")
  .attr("x", "74%")
  .attr("y", "76%")
  .attr("dy", "2.5em")
  .attr("font-size", "10px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 100)
  .text("in the new Academic Year 2020-2021.");

  svg.append("path")
    .attr('d', d3.line()([[635, 680], [570, 680]]))
    .attr("stroke-width", 2)
    .attr("stroke", "#989898")
    .attr("marker-end", "url(#triangle)");
});

});
'''

h = display(HTML(htmlt4))
j = py_display.Javascript(js_t4)
py_display.display_javascript(j)

<h3 style="font-family: times-new-roman">🇺🇸 States Engagement</h3>

<ul style="font-family: times-new-roman">
  <li>The <b>biggest engagement</b> registered in 2020 is in <b>North Dakota and Arizona</b> (however, we need to keep in mind that these states had data available from only one district).</li>
  <li><b>Connecticut and Massachusetts</b> are the states with the <b>biggest true engagement</b>, based on the fact that there was data collected from more than 20 districts for each of these states.</li>
  <li>We <i>can't say that a region in US is more active than another</i>, as the available states are quite dispersed throughout the country.</li>
</ul>

In [ ]:
# Prepare the data
us = pd.merge(df, districts, on="district_id", how="inner")

us = us[us["holiday"] == 0]\
    .groupby(["time", "lp_id", "state", "Abbreviation"])["engagement_index"]\
    .median().reset_index()\
    .groupby(["state", "Abbreviation"])["engagement_index"]\
    .median().reset_index()

# Map the rest of the states
mapping = pd.read_csv("../input/usa-states-to-region/states.csv")
us = pd.merge(mapping, us, left_on="State", 
         right_on="state", how="left").\
        drop(columns=["Region", "Division", "state", "Abbreviation"]).\
        fillna(0)
us.columns = ["state","Abbreviation","engagement_index"]

us.to_csv("us.csv", index=False)

# W&B run + save info
run = wandb.init(project='C19-learn', name='state_engagement', 
                 config=CONFIG, anonymous="allow")

create_wandb_plot(x_data=us["Abbreviation"], 
                  y_data=us["engagement_index"], 
                  x_name="State", y_name="Engagement", 
                  title="2020 States Engagement",
                  log="eng_state", plot="bar")

wandb.finish()

htmlt5 = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.andradutza {
		font-family: "Times New Roman", Times, serif;
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	line#limit {
	stroke: #b4c95a;
	stroke-width: 3;
	stroke-dasharray: 8 5;
	}

	text.divergence {
	font-size: 14px;
	fill: #000000;
	}

	text.value {
	font-size: 20px;
	fill: #000000;
	}

	.slider {
	-webkit-appearance: none; 
	appearance: none;
	width: 10%; 
	height: 15px; 
	background: #e7e7e7; 
	outline: none;
	opacity: 0.7; 
	border-radius: 6px;
	-webkit-transition: .2s; 
	transition: opacity .2s;
	}

	/* Mouse-over effects */
	.slider:hover {
	opacity: 1;
	}

	.slider::-webkit-slider-thumb {
	-webkit-appearance: none; /* Override default look */
	appearance: none;
	width: 15px; /* Set a specific slider handle width */
	height: 15px; /* Slider handle height */
	background: #1ACEE6; /* Green background */
	cursor: round; /* Cursor on hover */
	border-radius: 6px;
	}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 0.8rem;
        z-index: 99999;
	}

</style>

<img id="baseimg" src="usa.png" style="display:none" />
<div class="andradutza" style="background-color:#474747;">
	<center><h1 style="color:white;">2020 States Engagement</h1></center>
	<center><h4 style="color:white;">-Median Product Engagement in a Year-</h4></center>
	<center><h6 style="color:white;">(excluding legal holidays)</h6></center>
    <div class="row">
		<div id="map4" class="svg-container"></div>
	</div>
</div>
'''


js_t5 = '''
require.config({
  paths: {
    d3v6: "https://d3js.org/d3.v6.min",
    moment: "https://unpkg.com/moment@2.29.1/moment",
  }
});

require(["d3v6", "moment"], function(d3, moment) {

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/usa.png.*$/, '');

// Set the SVG area
const margin = { LEFT: 30, RIGHT: 30, TOP: 10, BOTTOM: 30 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM

const svg = d3.select("#map4").append("svg")
  //.attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)



// Map and projection
const projection = d3.geoMercator()
    .scale(730) // This is the zoom
    .translate([width+870, height]); // You have to play with these values to center your map

// Path generator
const path = d3.geoPath().projection(projection)

// Data and color scale
const data = new Map();
const colorScale = d3.scaleSequential()
      .domain([0, 5])
      .interpolator(d3.interpolate("#b2ef91", "#fa9372"));

// === Map ===
Promise.all([
  d3.json("https://raw.githubusercontent.com/holtzy/D3-graph-gallery/master/DATA/us_states_hexgrid.geojson.json"),
  d3.csv(baseAssetsUrl + "us.csv", function(d) {
      data.set(d.Abbreviation, +d.engagement_index)
  })]).then(function(loadData){

  let topo = loadData[0]

  // Interactive
  var div = d3.select("body").append("div")
    .attr("class", "tooltip-donut")
    .style("opacity", 0);

  let mouseOver = function(event, d) {
    d3.selectAll(".Country")
      .transition()
      .duration(100)
      .style("opacity", .5)
    d3.select(this)
      .transition()
      .duration(100)
      .style("opacity", 1)

    div.transition()
    .duration(100)
    .style("opacity", 1);
    let num = data.get(d.properties.iso3166_2);
    div.html(num)
        .style("left", (event.pageX) + "px")
        .style("top", (event.pageY-30) + "px");
  }

  let mouseLeave = function(d) {
    d3.selectAll(".Country")
      .transition()
      .duration(100)
      .style("opacity", 1)
    d3.select(this)
      .transition()
      .duration(100)

    // New div dissapears
    div.transition()
      .duration(100)
      .style("opacity", 0);
  }

  // Draw the map
  svg.append("g")
      .selectAll("path")
      .data(topo.features)
      .enter()
      .append("path")
      .attr("d", path)
      .attr("fill", function (d) {
        d.total = data.get(d.properties.iso3166_2);
        return colorScale(d.total);
      })
      .style("stroke", "#474747")
      .attr("stroke-width", 4)
      .attr("class", function(d){ return "Country" } )
      .style("opacity", 1)
      .on("mouseover", mouseOver )
      .on("mouseleave", mouseLeave )

  // Add the labels
  svg.append("g")
      .selectAll("labels")
      .data(topo.features)
      .join("text")
        .attr("x", function(d){return path.centroid(d)[0]})
        .attr("y", function(d){return path.centroid(d)[1]})
        .text(function(d){ return d.properties.iso3166_2})
        .attr("text-anchor", "middle")
        .attr("alignment-baseline", "central")
        .style("font-size", 16)
        .style("fill", "474747")
        .style("pointer-events", "none")
        .attr("font-weight", 600)
})


// === Bar ===
d3.csv(baseAssetsUrl + "states.csv").then(function(data){ 
  
  data.forEach(d => {
    d.NoDistricts = Number(d.NoDistricts)
  })

  // Gradient
  const defs = svg.append('defs');

  const bgGradient = defs
    .append('linearGradient')
    .attr('id', 'bg-gradient2')
    .attr('gradientTransform', 'rotate(90)');
  bgGradient
    .append('stop')
    .attr('stop-color', '#b4c95a') 
    .attr('offset', '0%');
  bgGradient
    .append('stop')
    .attr('stop-color', '#1acee6')
    .attr('offset', '100%');

  // Title
  var title = svg.append("text")
	.attr("y", "64%")
	.attr("x", "47%")
	.attr("font-size", "18px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .attr("fill", "white")
  .text("Frequency of Districts per State")

  // Scales
  const x = d3.scaleBand()
    .range([0, width/1.1])
    .domain(data.map(d => d.State))
    .paddingInner(0.3)
    .paddingOuter(0.2)

  const y = d3.scaleLinear()
    .range([height/5, 0])
    .domain([0, d3.max(data, d => d.NoDistricts)])

  const xAxisGroup = svg.append("g")
    .attr("class", "x axis")
    .attr("transform", `translate(60, ${width/1.14})`)

  const yAxisGroup = svg.append("g")
    .attr("class", "y axis")
    .attr("transform", `translate(50, ${width/1.48})`)

  // Axis
  const xAxisCall = d3.axisBottom(x)
  xAxisGroup.call(xAxisCall)
    .selectAll("text")
      .attr("y", "0")
      .attr("x", "-10")
      .attr("fill", "white")
      .attr("font-size", "13px")
      .attr("text-anchor", "end")
      .attr("transform", "rotate(-50)")

  const yAxisCall = d3.axisLeft(y)
    .ticks(7)
    .tickFormat(d => d)
  yAxisGroup.call(yAxisCall)
    .selectAll("text")
    .attr("fill", "white")

  // Bar Chart
  const rects = svg.selectAll()
    .data(data)
    .enter().append('g')
    .append("rect")
    .attr("transform", `translate(60, ${width/1.48})`)
    .attr("y", d => y(d.NoDistricts))
    .attr("x", d => x(d.State))
    .attr('rx', 5)
    .attr('ry', 4)
    .attr("width", x.bandwidth)
    .attr("height", d => height/5 - y(d.NoDistricts))
    .style("fill", "url(#bg-gradient2)")
    .style("stroke-width", "1px")


  // Interactive
  var div = d3.select("body").append("div")
    .attr("class", "tooltip-donut")
    .style("opacity", 0);

  rects
    .on('mouseenter', function (event, dt) {
    // MOUSE ON
    d3.selectAll('.value')
          .attr('opacity', 0)

    d3.select(this)
        .transition()
        .duration(100)
        .attr('opacity', 0.3)
        .attr('x', (a) => x(a.State) - 2.5)
        .attr('width', x.bandwidth() + 5)

    // Add line
    const y_new = y(dt.NoDistricts)
    var line = svg.append('line')
          .attr('id', 'limit')
          .attr('x1', 50)
          .attr('y1', y_new + 600)
          .attr('x2', 865)
          .attr('y2', y_new + 600)
          .attr('stroke', '#b4c95a')

    // Makes the new div appear
    div.transition()
        .duration(50)
        .style("opacity", 1);
    let num = dt.NoDistricts;
    div.html(num)
        .style("left", (event.pageX) + "px")
        .style("top", (event.pageY-30) + "px");


  })
    // MOUSE LEAVE
    .on('mouseleave', function () {
      d3.selectAll('.value')
          .attr('opacity', 1)

      d3.select(this)
      .transition()
      .duration(100)
      .attr('opacity', 1)
      .attr('x', (a) => x(a.State))
      .attr('width', x.bandwidth())

      // New div dissapears
      div.transition()
          .duration('50')
          .style("opacity", 0);

      svg.selectAll('#limit').remove()
      svg.selectAll('.divergence').remove()
    })

});

});
'''

h = display(HTML(htmlt5))
j = py_display.Javascript(js_t5)
py_display.display_javascript(j)

<h3 style="font-family: times-new-roman">🚸 Is there an Engagement Gap?</h3>

<ul style="font-family: times-new-roman">
  <li>On average, there is <b>lower engagement in the states that have a majority of black or hispanic students</b> vs the ones that have only a partial number (less than 50%). This trend is mantained ~equal throughout the entire year.</li>
  <li>There is a <b>lower engagement in the states where the majority of students receive free or reduced lunch</b> vs the states were less than 50% of them do. However, we can observe that this difference narrows in the second part of the year.</li>
  <li>A very interesting finding is that, although <b>children in states with big expenditure</b> per pupil have registered bigger engagement, there is <b>no significantly big difference</b> between them and <b>states where the expenditure per pupil is much lower</b>. Moreover, the difference narrows even more in the second half of the year.</li>
</ul>

In [ ]:
# Compute the Median
gap = df.groupby(["time", "district_id"])["engagement_index"]\
            .median().reset_index()

# Add States
gap = pd.merge(gap, districts, on="district_id").iloc[:, :4]

# Add Characteristics
to_keep = ['State', 'BlackHispanic', 'FreeReducedLunch', 'PerPupilExpenditure']
gap = pd.merge(gap, characteristics[to_keep], 
               left_on="state", right_on="State").drop(columns="state")

# Get month
gap["month"] = gap["time"].apply(lambda x: calendar.month_abbr[int(x.split("-")[1])])
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
          "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
gap['month'] = pd.Categorical(gap['month'], categories=months, ordered=True)


# Compute the 3 dataframes for each characteristic
col_chars = ['BlackHispanic', 'FreeReducedLunch', 'PerPupilExpenditure']
all_gaps = []

for c in col_chars:
    g = gap.groupby(["month", c])["engagement_index"].median().reset_index()
    g = g.pivot(index="month", columns=c, 
                values="engagement_index").reset_index()
    all_gaps.append(g)

gap0 = all_gaps[0]
gap1 = all_gaps[1].drop(columns="Unclear")
gap2 = all_gaps[2].drop(columns="Unclear")

# Save
gap0.to_csv("gap0.csv", index=False)
gap1.to_csv("gap1.csv", index=False)
gap2.to_csv("gap2.csv", index=False)


htmlt9 = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.andradutza {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisWhite path{
	stroke: #ffffff;
	}

	.axisWhite line{
	stroke: #ffffff;
	}

	.axisHidden path{
	stroke: #474747;
	}

	.axisHidden line{
	stroke: #474747;
	}

	.annotation.green text {
		fill: #a9ff68;
	}
	.annotation.red text {
		fill: #ff8989;
	}

</style>

<img id="baseimg" src="usa.png" style="display:none" />
<div class="andradutza" style="background-color:#474747;">
	<center><h1 style="color:white;">🚸 Is there an Engagement Gap?</h1></center>
	<center><h4 style="color:white;">-Median Engagement on District Characteristics-</h4></center>
	<div class="row">
		<div id="map9" class="svg-container"></div>
	</div>
</div>
'''


js_t9 = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/usa.png.*$/, '');

// Set the SVG area
const margin = { LEFT: 30, RIGHT: 30, TOP: 10, BOTTOM: 30 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM

const svg = d3.select("#map9").append("svg")
  // .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)


const color1 = "#ff8989";
const color2 = "#a9ff68";


// === Chart 0 ===
d3.csv(baseAssetsUrl + "gap0.csv").then(function(data){ 

  // Title
  svg.append("text")
  .attr("y", "9%")
	.attr("x", "18%")
	.attr("font-size", "20px")
	.attr("text-anchor", "middle")
  .attr("font-weight", 900)
  .attr("fill", "white")
  .text("Black/Hispanic")

  const defs = svg.append('defs');

  const bgGradient = defs
    .append('linearGradient')
    .attr('id', 'bg-gradient4')
    .attr('gradientTransform', 'rotate(90)');
  bgGradient
    .append('stop')
    .attr('stop-color', color1)
    .attr('offset', '0%');
  bgGradient
    .append('stop')
    .attr('stop-color', color2)
    .attr('offset', '100%');

  // Add X axis
  const x = d3.scaleLinear()
    .domain([0, 4.5])
    .range([ 0, width/3.5]);
  svg.append("g")
    .attr("class", "axisWhite")
    .attr("transform", `translate(50, ${height/1})`)
    .call(d3.axisBottom(x))
    .selectAll("text")
    .attr("font-size", "11px")
    .attr("fill", "white")
    .attr("font-weight", 600)

  // Y axis
  const y = d3.scaleBand()
    .range([ 0, height/1.09])
    .domain(data.map(function(d) { return d.month; }))
    .padding(1);
  svg.append("g")
  .attr("class", "axisWhite")
  .attr("transform", `translate(30, ${width/3-227})`)
    .call(d3.axisLeft(y))
    .selectAll("text")
    .attr("font-size", "11px")
    .attr("fill", "white")
    .attr("font-weight", 600)


  // Lines
  svg.selectAll("myline")
    .data(data)
    .join("line")
    .attr("transform", `translate(50, ${width/3-227})`)
      .attr("x1", function(d) { return x(d.Maximum); })
      .attr("x2", function(d) { return x(d.Partial); })
      .attr("y1", function(d) { return y(d.month); })
      .attr("y2", function(d) { return y(d.month); })
      // .style("stroke", "url(#bg-gradient4)")
      .style("stroke", "grey")
      .attr("stroke-width", "3px")

  // Circles of variable 1
  svg.selectAll("mycircle")
    .data(data)
    .join("circle")
    .attr("transform", `translate(50, ${width/3-227})`)
      .attr("cx", function(d) { return x(d.Maximum); })
      .attr("cy", function(d) { return y(d.month); })
      .attr("r", "7")
      .style("fill", color1)
      

  // Circles of variable 2
  svg.selectAll("mycircle")
    .data(data)
    .join("circle")
    .attr("transform", `translate(50, ${width/3-227})`)
      .attr("cx", function(d) { return x(d.Partial); })
      .attr("cy", function(d) { return y(d.month); })
      .attr("r", "7")
      .style("fill", color2)
  
});


// === Chart 1 ===
d3.csv(baseAssetsUrl + "gap1.csv").then(function(data){ 

  // Title
  svg.append("text")
  .attr("y", "9%")
	.attr("x", "49%")
	.attr("font-size", "20px")
	.attr("text-anchor", "middle")
  .attr("font-weight", 900)
  .attr("fill", "white")
  .text("Free/Reduced Lunch")

  const defs = svg.append('defs');

  const bgGradient = defs
    .append('linearGradient')
    .attr('id', 'bg-gradient4')
    .attr('gradientTransform', 'rotate(90)');
  bgGradient
    .append('stop')
    .attr('stop-color', color1)
    .attr('offset', '0%');
  bgGradient
    .append('stop')
    .attr('stop-color', color2)
    .attr('offset', '100%');

  // Add X axis
  const x = d3.scaleLinear()
    .domain([0, 4.5])
    .range([ 0, width/3.5]);
  svg.append("g")
    .attr("class", "axisWhite")
    .attr("transform", `translate(340, ${height/1})`)
    .call(d3.axisBottom(x))
    .selectAll("text")
    .attr("font-size", "11px")
    .attr("fill", "white")
    .attr("font-weight", 600)

  // Y axis
  const y = d3.scaleBand()
    .range([ 0, height/1.09])
    .domain(data.map(function(d) { return d.month; }))
    .padding(1);
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(340, ${width/3-227})`)
    .call(d3.axisLeft(y))
    .selectAll("text")
    .attr("font-size", "11px")
    .attr("fill", "#474747")
    .attr("font-weight", 600)


  // Lines
  svg.selectAll("myline")
    .data(data)
    .join("line")
    .attr("transform", `translate(340, ${width/3-227})`)
      .attr("x1", function(d) { return x(d.Maximum); })
      .attr("x2", function(d) { return x(d.Partial); })
      .attr("y1", function(d) { return y(d.month); })
      .attr("y2", function(d) { return y(d.month); })
      // .style("stroke", "url(#bg-gradient4)")
      .style("stroke", "grey")
      .attr("stroke-width", "3px")

  // Circles of variable 1
  svg.selectAll("mycircle")
    .data(data)
    .join("circle")
    .attr("transform", `translate(340, ${width/3-227})`)
      .attr("cx", function(d) { return x(d.Maximum); })
      .attr("cy", function(d) { return y(d.month); })
      .attr("r", "7")
      .style("fill", color1)
      

  // Circles of variable 2
  svg.selectAll("mycircle")
    .data(data)
    .join("circle")
    .attr("transform", `translate(340, ${width/3-227})`)
      .attr("cx", function(d) { return x(d.Partial); })
      .attr("cy", function(d) { return y(d.month); })
      .attr("r", "7")
      .style("fill", color2)
  
});


// === Chart 2 ===
d3.csv(baseAssetsUrl + "gap2.csv").then(function(data){ 

  // Title
  svg.append("text")
  .attr("y", "9%")
	.attr("x", "80%")
	.attr("font-size", "20px")
	.attr("text-anchor", "middle")
  .attr("font-weight", 900)
  .attr("fill", "white")
  .text("Per Pupil Expenditure")

  const defs = svg.append('defs');

  const bgGradient = defs
    .append('linearGradient')
    .attr('id', 'bg-gradient4')
    .attr('gradientTransform', 'rotate(90)');
  bgGradient
    .append('stop')
    .attr('stop-color', color1)
    .attr('offset', '0%');
  bgGradient
    .append('stop')
    .attr('stop-color', color2)
    .attr('offset', '100%');

  // Add X axis
  const x = d3.scaleLinear()
    .domain([0, 4.5])
    .range([ 0, width/3.5]);
  svg.append("g")
    .attr("class", "axisWhite")
    .attr("transform", `translate(630, ${height/1})`)
    .call(d3.axisBottom(x))
    .selectAll("text")
    .attr("font-size", "11px")
    .attr("fill", "white")
    .attr("font-weight", 600)

  // Y axis
  const y = d3.scaleBand()
    .range([ 0, height/1.09])
    .domain(data.map(function(d) { return d.month; }))
    .padding(1);
  svg.append("g")
  .attr("class", "axisHidden")
  .attr("transform", `translate(630, ${width/3-227})`)
    .call(d3.axisLeft(y))
    .selectAll("text")
    .attr("font-size", "11px")
    .attr("fill", "#474747")
    .attr("font-weight", 600)


  // Lines
  svg.selectAll("myline")
    .data(data)
    .join("line")
    .attr("transform", `translate(630, ${width/3-227})`)
      .attr("x1", function(d) { return x(d.Maximum); })
      .attr("x2", function(d) { return x(d.Partial); })
      .attr("y1", function(d) { return y(d.month); })
      .attr("y2", function(d) { return y(d.month); })
      // .style("stroke", "url(#bg-gradient4)")
      .style("stroke", "grey")
      .attr("stroke-width", "3px")

  // Circles of variable 1
  svg.selectAll("mycircle")
    .data(data)
    .join("circle")
    .attr("transform", `translate(630, ${width/3-227})`)
      .attr("cx", function(d) { return x(d.Maximum); })
      .attr("cy", function(d) { return y(d.month); })
      .attr("r", "7")
      .style("fill", color1)
      

  // Circles of variable 2
  svg.selectAll("mycircle")
    .data(data)
    .join("circle")
    .attr("transform", `translate(630, ${width/3-227})`)
      .attr("cx", function(d) { return x(d.Partial); })
      .attr("cy", function(d) { return y(d.month); })
      .attr("r", "7")
      .style("fill", color2)
  
});


// Legend
const annot1 = [
  {
  note: { 
    label: "(< 50%)",
    title: "Partial",
    wrap: 230,  // smaller text splits in multiple lines
    padding: 0,   // More = text lower
  },
  className: "green",
  type: d3.annotationCalloutCircle,
  subject: { radius: 12 },
  color: ["#ffffff"],
  x: 198,
  y: 130,
  dy: -20,
  dx: 50
},
{
  note: { 
    label: "(> 50%)",
    title: "Majority",
    wrap: 230,  // smaller text splits in multiple lines
    padding: 0,   // More = text lower
  },
  className: "red",
  type: d3.annotationCalloutCircle,
  subject: { radius: 12 },
  color: ["#ffffff"],
  x: 160.5,
  y: 251.5,
  dy: -20,
  dx: -35
},
{
  note: { 
    label: "(< 50%)",
    title: "Partial",
    wrap: 230,  // smaller text splits in multiple lines
    padding: 0,   // More = text lower
  },
  className: "green",
  type: d3.annotationCalloutCircle,
  subject: { radius: 12 },
  color: ["#ffffff"],
  x: 553,
  y: 312,
  dy: -20,
  dx: 40
},
{
  note: { 
    label: "(> 50%)",
    title: "Majority",
    wrap: 230,  // smaller text splits in multiple lines
    padding: 0,   // More = text lower
  },
  className: "red",
  type: d3.annotationCalloutCircle,
  subject: { radius: 12 },
  color: ["#ffffff"],
  x: 419,
  y: 373,
  dy: -20,
  dx: -20
},
{
  note: { 
    label: "(< $1,000/pupil)",
    title: "Small Expenditure",
    wrap: 230,  // smaller text splits in multiple lines
    padding: 0,   // More = text lower
  },
  className: "green",
  type: d3.annotationCalloutCircle,
  subject: { radius: 12 },
  color: ["#ffffff"],
  x: 755,
  y: 676,
  dy: -20,
  dx: -20
},
{
  note: { 
    label: "(> $1,000/pupil)",
    title: "Big Expenditure",
    wrap: 230,  // smaller text splits in multiple lines
    padding: 0,   // More = text lower
  },
  className: "red",
  type: d3.annotationCalloutCircle,
  subject: { radius: 12 },
  color: ["#ffffff"],
  x: 757,
  y: 494,
  dy: -15,
  dx: 20
}]

svg.append("g")
  .style('font-size', 14)
  .call(d3.annotation()
          .annotations(annot1))

});
'''

h = display(HTML(htmlt9))
j = py_display.Javascript(js_t9)
py_display.display_javascript(j)

<h2 style="font-family: times-new-roman">2.2 Learning Products</h2>

<h3 style="font-family: times-new-roman"> 🍎 Top Products Overview</h3>

<ul style="font-family: times-new-roman">
  <li>Most used educational products in 2020 were <b>Google products</b> (like Google Docs, Google Classroom, Google Drive, Google Forms, Google Sheets, Google Meet), <b>Youtube</b> and <b>Kahoot!</b>.</li>
  <li>Compared to these few products, all the others had extremely little to no median engagement throughout the year. Hence, Google and Kahoot! have <i>monopol</i> so far over the online educational instruments.</li>
</ul>

<div class="alert simple-alert" style="font-family: times-new-roman">
  <p style="font-family: times-new-roman"><b>📌 Note</b>: This could be seen as an opportunity for the other e-learning products to increase their reach and marketing. The reason that these products aren't used is not that they aren't "good enough" - is because teachers do not have the time and sometimes the ability to research these products, so they stick with what they know already. I know for a fact from my mother that teachers talk a lot between them and share tips, but if most don't get outside the bubble often enough they can lose thack of new or already available products that might help and improve their way of teaching online.</p>
</div>

In [ ]:
# Get product ids
n = 25
top_n = df.groupby(["lp_id"])["engagement_index"].mean().reset_index().\
            sort_values("engagement_index", ascending=False).\
            head(n)["lp_id"].values

# Transform the data
proverview = df[df["holiday"] == 0].\
                groupby(["lp_id"])["engagement_index"].median().\
                reset_index()
proverview = pd.merge(proverview, prods[["LP ID", "Product Name"]], 
                     left_on="lp_id", right_on="LP ID")
proverview = proverview[proverview["lp_id"].isin(top_n)]

proverview = proverview.drop(columns=["lp_id", "LP ID"])

# Beautify
proverview.columns = ["engagement_index", "name"]
proverview = proverview.sort_values("engagement_index", ascending=False).\
                        reset_index(drop=True)
proverview["engagement_index"] = proverview["engagement_index"].astype("int64")

# Save to .csv
proverview.to_csv("proverview.csv", index=False)

# Save images as well
im = imageio.imread('../input/covid19-education-data/googledocs.png')
Image.fromarray(im).save("googledocs.png")
im = imageio.imread('../input/covid19-education-data/youtube.png')
Image.fromarray(im).save("youtube.png")
im = imageio.imread('../input/covid19-education-data/googleclassroom.png')
Image.fromarray(im).save("googleclassroom.png")
im = imageio.imread('../input/covid19-education-data/googledrive.png')
Image.fromarray(im).save("googledrive.png")
im = imageio.imread('../input/covid19-education-data/kahoot.png')
Image.fromarray(im).save("kahoot.png")

# W&B run + save info
run = wandb.init(project='C19-learn', name='top_prods', 
                 config=CONFIG, anonymous="allow")

create_wandb_plot(x_data=proverview["name"], y_data=proverview["engagement_index"], 
                  x_name="Product Name", y_name="Median Engagement", 
                  title="Educational Products Overview",
                  log="top_prods", plot="bar")
wandb.finish()


htmlt7 = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.andradutza {
		font-family: "Times New Roman", Times, serif;
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	line#limit {
	stroke: #f7b227;
	stroke-width: 2;
	stroke-dasharray: 8 5;
	}

	text.divergence {
	font-size: 14px;
	fill: #000000;
	}

	text.value {
	font-size: 20px;
	fill: #000000;
	}

	.slider {
	-webkit-appearance: none; 
	appearance: none;
	width: 10%; 
	height: 15px; 
	background: #e7e7e7; 
	outline: none;
	opacity: 0.7; 
	border-radius: 6px;
	-webkit-transition: .2s; 
	transition: opacity .2s;
	}

	/* Mouse-over effects */
	.slider:hover {
	opacity: 1;
	}

	.slider::-webkit-slider-thumb {
	-webkit-appearance: none; /* Override default look */
	appearance: none;
	width: 15px; /* Set a specific slider handle width */
	height: 15px; /* Slider handle height */
	background: #1ACEE6; /* Green background */
	cursor: round; /* Cursor on hover */
	border-radius: 6px;
	}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 0.8rem;
	}

</style>

<img id="baseimg" src="usa.png" style="display:none" />
<div class="andradutza">
	<center><h1>Educational Products Overview</h1></center>
	<center><h4>-Median Product Engagement in a Year-</h4></center>
	<center><h6>(excluding legal holidays)</h6></center>
    <div class="row">
		<div id="map7" class="svg-container"></div>
	</div>
</div>
'''


js_t7 = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/usa.png.*$/, '');

// Set the SVG area
const margin = { LEFT: 30, RIGHT: 30, TOP: 10, BOTTOM: 30 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM

const svg = d3.select("#map7").append("svg")
  // .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)


// === Bar ===
d3.csv(baseAssetsUrl + "proverview.csv").then(function(data){ 

  data.forEach(d => {
    d.engagement_index = Number(d.engagement_index)
  })

  const defs = svg.append('defs');

  const bgGradient = defs
    .append('linearGradient')
    .attr('id', 'bg-gradient3')
    .attr('gradientTransform', 'rotate(90)');
  bgGradient
    .append('stop')
    .attr('stop-color', '#9bf8f4')
    .attr('offset', '0%');
  bgGradient
    .append('stop')
    .attr('stop-color', '#6f7bf7')
    .attr('offset', '100%');

  // Scales
  const x = d3.scaleBand()
    .range([0, width/1.1])
    .domain(data.map(d => d.name))
    .paddingInner(0.3)
    .paddingOuter(0.2)

  const y = d3.scaleLinear()
    .range([height/4, 0])
    .domain([0, d3.max(data, d => d.engagement_index)])

  const xAxisGroup = svg.append("g")
    .attr("class", "x axis")
    .attr("transform", `translate(60, ${width/1.86})`)

  const yAxisGroup = svg.append("g")
    .attr("class", "y axis")
    .attr("transform", `translate(50, ${width/3.5})`)

  // Axis
  const xAxisCall = d3.axisBottom(x)
  xAxisGroup.call(xAxisCall)
    .selectAll("text")
      .attr("y", "0")
      .attr("x", "-10")
      .attr("text-anchor", "end")
      .attr("font-size", "12px")
      .attr("transform", "rotate(-50)")

  const yAxisCall = d3.axisLeft(y)
    .ticks(7)
    .tickFormat(d => d)
  yAxisGroup.call(yAxisCall)
    .selectAll("text")

  // Bar Chart
  const rects = svg.selectAll()
    .data(data)
    .enter().append('g')
    .append("rect")
    .attr("transform", `translate(50, ${width/3.5})`)
    .attr("y", d => y(d.engagement_index))
    .attr("x", d => x(d.name))
    .attr('rx', 5)
    .attr('ry', 4)
    .attr("width", x.bandwidth)
    .attr("height", d => height/4 - y(d.engagement_index))
    .style("fill", "url(#bg-gradient3)")
    .style("stroke-width", "1px")


  // Interactive
  var div = d3.select("body").append("div")
    .attr("class", "tooltip-donut")
    .style("opacity", 0);

  rects
    .on('mouseenter', function (event, dt) {
    // MOUSE ON
    d3.selectAll('.value')
          .attr('opacity', 0)

    d3.select(this)
        .transition()
        .duration(100)
        .attr('opacity', 0.3)
        .attr('x', (a) => x(a.name) - 2.5)
        .attr('width', x.bandwidth() + 5)

    // Add line
    const y_new = y(dt.engagement_index)
    var line = svg.append('line')
          .attr('id', 'limit')
          .attr('x1', 50)
          .attr('y1', y_new + 250)
          .attr('x2', 850)
          .attr('y2', y_new + 250)
          .attr('stroke', '#f7b227')

    // Makes the new div appear
    div.transition()
        .duration(50)
        .style("opacity", 1);
    let num = dt.engagement_index;
    div.html(num)
        .style("left", (event.pageX) + "px")
        .style("top", (event.pageY-30) + "px");


  })
    // MOUSE LEAVE
    .on('mouseleave', function () {
      d3.selectAll('.value')
          .attr('opacity', 1)

      d3.select(this)
      .transition()
      .duration(100)
      .attr('opacity', 1)
      .attr('x', (a) => x(a.name))
      .attr('width', x.bandwidth())

      // New div dissapears
      div.transition()
          .duration('50')
          .style("opacity", 0);

      svg.selectAll('#limit').remove()
      svg.selectAll('.divergence').remove()
    })


  // Google Docs
  svg.append("image")
  .attr("xlink:href", baseAssetsUrl + "googledocs.png")
  .attr("x", "5%")
  .attr("y", "1%")
  .attr("width", 50)
  .attr("height", 50)
  .style("opacity", 1);

  svg.append("text")
      .attr("x", "7.5%")
      .attr("y", "8.5%")
      .attr("font-size", "14px")
      .attr("text-anchor", "middle")
      .attr("font-weight", 600)
      .text("Google Docs")
  svg.append("text")
      .attr("x", "8.5%")
      .attr("y", "8.5%")
      .attr("dy", "1.5em")
      .attr("font-size", "11.5px")
      .attr("text-anchor", "middle")
      .attr("font-weight", 100)
      .text("Create and edit text docs")
  svg.append("text")
      .attr("x", "8.5%")
      .attr("y", "8.5%")
      .attr("dy", "2.5em")
      .attr("font-size", "11.5px")
      .attr("text-anchor", "middle")
      .attr("font-weight", 100)
      .text("right in your web browser.")

   svg.append("svg:defs").append("svg:marker")
   .attr("id", "triangle")
   .attr("refX", 6)
   .attr("refY", 6)
   .attr("markerWidth", 30)
   .attr("markerHeight", 30)
   .attr("markerUnits","userSpaceOnUse")
   .attr("orient", "auto")
   .append("path")
   .attr("d", "M 0 0 12 6 0 12 3 6")
   .style("fill", "#4E4E4E");
   svg.append("path")
   // bottom, top
     .attr('d', d3.line()([[75, 120], [75, 240]]))
     .attr("stroke-width", 2)
     .attr("stroke", "#4E4E4E")
     .attr("marker-end", "url(#triangle)");


  // Youtube
  svg.append("image")
  .attr("xlink:href", baseAssetsUrl + "youtube.png")
  .attr("x", "5%")
  .attr("y", "82%")
  .attr("width", 150)
  .attr("height", 150)
  .style("opacity", 1);
      
  svg.append("text")
  .attr("x", "13%")
  .attr("y", "94.5%")
  .attr("font-size", "14px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Youtube")
svg.append("text")
  .attr("x", "13.5%")
  .attr("y", "94.5%")
  .attr("dy", "1.5em")
  .attr("font-size", "11.5px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 100)
  .text("Provides various teaching videos")
svg.append("text")
  .attr("x", "13.5%")
  .attr("y", "94.5%")
  .attr("dy", "2.5em")
  .attr("font-size", "11.5px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 100)
  .text("and helps the students learn.")

svg.append("path")
// bottom, top
  .attr('d', d3.line()([[120, 780], [120, 600]]))
  .attr("stroke-width", 2)
  .attr("stroke", "#4E4E4E")
  .attr("marker-end", "url(#triangle)");


// Classroom
svg.append("image")
.attr("xlink:href", baseAssetsUrl + "googleclassroom.png")
.attr("x", "15.5%")
.attr("y", "13%")
.attr("width", 60)
.attr("height", 60)
.style("opacity", 1);

svg.append("text")
  .attr("x", "18.5%")
  .attr("y", "21%")
  .attr("font-size", "14px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Google Classroom")
svg.append("text")
  .attr("x", "18.5%")
  .attr("y", "21%")
  .attr("dy", "1.5em")
  .attr("font-size", "11.5px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 100)
  .text("Your central place where")
svg.append("text")
.attr("x", "18.5%")
.attr("y", "21%")
.attr("dy", "2.5em")
.attr("font-size", "11.5px")
.attr("text-anchor", "middle")
.attr("font-weight", 100)
.text("teaching and learning")
svg.append("text")
  .attr("x", "18.5%")
  .attr("y", "21%")
  .attr("dy", "3.5em")
  .attr("font-size", "11.5px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 100)
  .text("come together.")

svg.append("path")
// bottom, top
  .attr('d', d3.line()([[170, 240], [170, 380]]))
  .attr("stroke-width", 2)
  .attr("stroke", "#4E4E4E")
  .attr("marker-end", "url(#triangle)");


// Drive
svg.append("image")
.attr("xlink:href", baseAssetsUrl + "googledrive.png")
.attr("x", "20%")
.attr("y", "73%")
.attr("width", 50)
.attr("height", 50)
.style("opacity", 1);

svg.append("text")
  .attr("x", "22%")
  .attr("y", "80%")
  .attr("font-size", "14px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Google Drive")
svg.append("text")
  .attr("x", "22%")
  .attr("y", "80%")
  .attr("dy", "1.5em")
  .attr("font-size", "11.5px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 100)
  .text("Save files online")
svg.append("text")
.attr("x", "22%")
.attr("y", "80%")
.attr("dy", "2.5em")
.attr("font-size", "11.5px")
.attr("text-anchor", "middle")
.attr("font-weight", 100)
.text("and access them")
svg.append("text")
  .attr("x", "22%")
  .attr("y", "80%")
  .attr("dy", "3.5em")
  .attr("font-size", "11.5px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 100)
  .text("anywhere on any device.")

svg.append("path")
// bottom, top
  .attr('d', d3.line()([[215, 650], [215, 550]]))
  .attr("stroke-width", 2)
  .attr("stroke", "#4E4E4E")
  .attr("marker-end", "url(#triangle)");

// Kahoot
svg.append("image")
.attr("xlink:href", baseAssetsUrl + "kahoot.png")
.attr("x", "25%")
.attr("y", "28%")
.attr("width", 50)
.attr("height", 50)
.style("opacity", 1);

svg.append("text")
  .attr("x", "27.7%")
  .attr("y", "35.5%")
  .attr("font-size", "14px")
  .attr("text-anchor", "middle")
  .attr("font-weight", 600)
  .text("Kahoot!")
svg.append("text")
  .attr("x", "27.7%")
  .attr("y", "35.5%")
  .attr("dy", "1.5em")
  .attr("font-size", "11.5")
  .attr("text-anchor", "middle")
  .attr("font-weight", 100)
  .text("Game-based learning platform.")

  svg.append("path")
// bottom, top
  .attr('d', d3.line()([[265, 350], [265, 440]]))
  .attr("stroke-width", 2)
  .attr("stroke", "#4E4E4E")
  .attr("marker-end", "url(#triangle)");
});

// Note
const annotations = [
  {
  note: { 
    label: "Google products (like Google Docs, Classroom, Drive, Forms, Sheets and Meet), Youtube and Kahoot! are the most frequently used products in one year. \t Worth mentioning is that all the other products have very small median engagement compared to the mentioned ones.",
    title: "📌 Note",
    wrap: 250,  // smaller text splits in multiple lines
    padding: 10,   // More = text lower
    
  },
  color: ["#000000"],
  x: 630,
  y: 30,
  dy: 0,
  dx: 0
}]

const makeAnnotations = d3.annotation()
  .annotations(annotations)

svg.append("g")
.call(makeAnnotations)

});
'''

h = display(HTML(htmlt7))
j = py_display.Javascript(js_t7)
py_display.display_javascript(j)

<h3 style="font-family: times-new-roman"> 🖍️ Other Products</h3>

<p style="font-family: times-new-roman"> There are many more e-learning products that seem yet to be discovered by the teachers/ pupils. Here are 164 products with the biggest one having the biggest median engagement to much smaller ones, with less median engagement:</p>

In [ ]:
n = 400
top_all = df.groupby(["lp_id"])["engagement_index"].mean().reset_index().\
            sort_values("engagement_index", ascending=False).\
            head(n)["lp_id"].values

# Transform the data
wordcloud = df[df["holiday"] == 0].\
                groupby(["lp_id"])["engagement_index"].median().\
                reset_index()
wordcloud = pd.merge(wordcloud, prods[["LP ID", "Product Name"]], 
                     left_on="lp_id", right_on="LP ID")
wordcloud = wordcloud[wordcloud["lp_id"].isin(top_all)]

wordcloud = wordcloud.drop(columns=["lp_id", "LP ID"])

# Beautify
wordcloud.columns = ["engagement_index", "name"]
wordcloud = wordcloud.sort_values("engagement_index", ascending=False).\
                        reset_index(drop=True)
wordcloud["engagement_index"] = wordcloud["engagement_index"].astype("int64")

# Log scale for better visual
wordcloud["engagement_index"] = np.log10(wordcloud["engagement_index"]).\
                                replace([np.inf, -np.inf], 0)*10
wordcloud["engagement_index"] = wordcloud["engagement_index"].apply(lambda x: int(x))
wordcloud.columns = ["size", "word"]

wordcloud.to_csv("wordcloud.csv", index=False)


htmlt10 = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.andradutza {
		font-family: "Times New Roman", Times, serif;
		overflow-x: hidden;
		width: 98%;
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

</style>

<img id="baseimg" src="usa.png" style="display:none" />
<div class="andradutza" style="background-color:#474747;">
	<center><h1 style="color:white;">🖍️ All e-learning products</h1></center>
	<center><h4 style="color:white;">-wordcloud based on engagement <i>(log10 scale)</i>-</h4></center>
	<div class="row">
		<div id="map10" class="svg-container"></div>
	</div>
</div>
'''


js_t10 = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v4': 'd3src/d3.v4.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
      'd3-cloud': 'https://cdn.jsdelivr.net/gh/holtzy/D3-graph-gallery@master/LIB/d3.layout.cloud.js',
    }
  }
});

require(["d3v4", "d3-annotation", "d3-cloud"], function(d3, d3Annotation, d3Cloud) {

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/usa.png.*$/, '');

// Set the SVG area
const margin = { LEFT: 30, RIGHT: 30, TOP: 10, BOTTOM: 30 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM

const svg = d3.select("#map10").append("svg")
  // .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)


// === Wordcloud ===
d3.csv(baseAssetsUrl + "wordcloud.csv", function(data){ 
  
  var myColor = d3.scaleOrdinal(["#fc5a62","#fc8f7c","#fecd6c","#e1fa73","#5cfdf5","#59e9fc","#6e85f8","#7243f4","#f773f0","#fa5884"]);

  function draw(words) {
    svg
      .append("g")
        .attr("transform", "translate(" + layout.size()[0] / 2 + "," + layout.size()[1] / 2 + ")")
        .selectAll("text")
          .data(words)
        .enter().append("text")
          .style("font-size", function(d) { return d.size; })
          .attr("font-weight", 900)
          .style("fill", (d, i) => myColor(i))
          .attr("text-anchor", "middle")
          .attr("transform", function(d) {
            return "translate(" + [d.x, d.y] + ") rotate(" + d.rotate + ")";
          })
          .text(function(d) { return d.text; });
        }

  var layout = d3Cloud()
  .size([width, height])
  .words(data.map(function(d) { return {text: d.word, size:d.size}; }))
  .padding(5)        //space between words
  .rotate(function() { return ~~(Math.random() * 2) * 90; })
  .fontSize(function(d) { return d.size*1.3; })      // font size of words
  .on("end", draw);
layout.start();
  
});

});
'''

h = display(HTML(htmlt10))
j = py_display.Javascript(js_t10)
py_display.display_javascript(j)

<h3 style="font-family: times-new-roman">🎒 Learning Product Engagement Timeline </h3>

<ul style="font-family: times-new-roman">
  <li>Based on the most engaging 10 products, we can see that there is an increase in the new school year 2020-2021 vs the engagement before the big summer break. This means that <b>students started not only getting used to these products, but also realised that they're here to stay and they need to start using them more often</b>.</li>
  <li>Youtube started increasing in engagement during summer 2020 and sustained it's activity in the new school year.</li>
  <li>Although after the start of the pandemic there has been considerable increase in usage for these e-learning tools, the <b>difference after vs the little time we have available before the pandemic isn't that substantial</b>. This could happen because most of the kids that were active after the pandemic started were also active using these tools before. However, there is a <b>big pool of students that seem to not use these products at all</b> (and this can be seen in the activity percentage).</li>
</ul>

In [ ]:
# Get product ids
n = 17
top_n = df.groupby(["lp_id"])["engagement_index"].mean().reset_index().\
            sort_values("engagement_index", ascending=False).\
            head(n)["lp_id"].values

# Transform the data
timeprod = df.groupby(["time", "lp_id"])["engagement_index"].median().\
                reset_index()
timeprod = pd.merge(timeprod, prods[["LP ID", "Product Name"]], 
                     left_on="lp_id", right_on="LP ID")
timeprod = timeprod[timeprod["lp_id"].isin(top_n)]

timeprod = pd.pivot(timeprod.drop(columns=["lp_id", "LP ID"]), index="time",
                     columns="Product Name", values="engagement_index").\
                    reset_index().fillna(0)

# Log scale for better visual
for c in timeprod.columns[1:]:
    timeprod[c] = np.log10(timeprod[c]).\
                    replace([np.inf, -np.inf], 0)
    timeprod[c] = timeprod[c]*100
    
# Convert from string to number
string_date = timeprod["time"]
timeprod.insert(0, 'time2', string_date)
timeprod["time"] = pd.to_datetime(timeprod["time"]).apply(lambda x: x.value)
for col in timeprod.columns[1:]: 
    timeprod[col] = timeprod[col].astype("int64")
    
# Save to .csv
timeprod.to_csv("timeprod.csv", index=False)



htmlt7 = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.andradutza {
		font-family: "Times New Roman", Times, serif;
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}
	.area-label {
        font-family: sans-serif;
        fill-opacity: 1;
        fill: white;
      }

</style>

<img id="baseimg" src="usa.png" style="display:none" />
<div class="andradutza">
	<center><h1>Top 10 Educational Products</h1></center>
	<center><h4>-Median Daily Student Engagement <i>(log10 scale)</i>-</h4></center>
    <div class="row">
		<div id="map6" class="svg-container"></div>
	</div>
</div>
'''


js_t7 = '''
require.config({
  paths: {
    d3v6: "https://d3js.org/d3.v6.min",
    moment: "https://unpkg.com/moment@2.29.1/moment",
  }
});

require(["d3v6", "moment"], function(d3, moment) {

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/usa.png.*$/, '');

// Set the SVG area
const margin = { LEFT: 100, RIGHT: 10, TOP: 10, BOTTOM: 100 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM
const chart_size = 1650

const svg = d3.select("#map6").append("svg")
  // .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)


svg.append("image")
.attr("xlink:href", baseAssetsUrl + "schoolbreak.png")
    .attr("x", "-3%")
    .attr("y", "41%")
    .attr("width", 240)
    .attr("height", 200)
    .style("opacity", 1);


d3.csv(baseAssetsUrl + "timeprod.csv").then((data) => {
  
  // List of groups = header of the csv files
  var keys = data.columns.slice(2)
  var days = ["Mar 16","Mar 24","Jun 12","Sep 10"];
  var formatDay = (d, i) => {
      return days[i];      
  }

  // Stacking
  var series = d3.stack()
    .keys(keys)
    .offset(d3.stackOffsetSilhouette)
    (data)

  // Area Chart
  var area = d3.area()
  .y(d => y(d.data.time))
  .x0(d => x(d[0]))
  .x1(d => x(d[1]))

  // X and Y axis
  var y = d3.scaleLinear()
    .domain(d3.extent(data, d => d.time))
    .range([0, width])

  var x = d3.scaleLinear()
  .domain([-chart_size, chart_size])
  .range([height, 0])

  // Color
  const color = d3.scaleOrdinal()
    .domain(keys)
    .range(["#db222a", "#e16853", "#f7b227", "#b4c95a",
            "#1be9df", "#1acee6", "#5d6dba", "#5c4d86",
            "#af6eac", "#c1385c"])

  // X axis placement
  var yAxis = g => g
    .attr("transform", "translate(0," + (height/2-1) + ")")
    .call(d3.axisLeft(y)
            .tickSize(-height+100)
            .tickValues([1584316800000000000,1585008000000000000,
              1591920000000000000, 1599696000000000000])
            .tickFormat(formatDay)
            )
    .call(g => g.select(".domain").remove())
    .call(g => g.selectAll(".tick line")
                .attr("stroke", "#989898")
                .attr('stroke-dasharray', '5 5'))
    .call(g => g.selectAll(".tick")
                .select('text')
                .attr('fill', "#989898")
                .style("font-size", 13))
                .attr("transform", "translate(50,0)")

  // Show
  const path = svg.append("g")
    .selectAll("path")
    .data(series)
    .join("path")
      .attr("data-genre", d => d.key)
      .attr("fill", ({key}) => color(key))
      .attr("d", area)
      .attr("opacity", 0.8)

  svg.append("g")
      .call(yAxis)

  // ======================================
  const hover = (svg, path) => {
  
    const line = svg.append("g")
         .attr("display", "none");
  
    line.append("g")
        .selectAll("line")
        .data(series)
        .join("line")
        .attr("class", "cursor-line")
        .attr("fill", "#fff")
        .attr("stroke-width", 0.8)
        .attr("y1", 10)
        .attr("x1", width-50)
        .attr("y2", 10)
        .attr("x2", 10);
  
    line.append("text")
        .attr("class", "text-year")
        .attr("font-size", 13)
        .attr("x", 0)
        .attr("y", 26)
        .attr("transform", 
        "rotate(0 0 0) translate(10, -20)");
    
    line.append("g")
        .selectAll("text")
        .data(series)
        .join("text")
        .attr("class", "text-genre")
        .attr("font-weight", "bold")
        .attr("font-size", 13)
        .attr("x", width/2+300)
        .attr("y", 26)
        .attr("transform", 
        "rotate(0 0 10)");

    line.append("g")
        .selectAll("text")
        .data(data)
        .join("text")
        .attr("class", "text-rating")
        .attr("font-size", 13)
        .attr("x", width/2+190)
        .attr("y", -5);

    const mousemove = (event) => {
  
      event?.preventDefault();
      // pointer = [posX, posY]
      const pointer = d3.pointer(event);
          
      const xm = x.invert(pointer[0]);
      const ym = y.invert(pointer[1]);
      const genre = d3.select(event.target).attr("data-genre");
  
      line.attr("transform", `translate(0 ${pointer[1]})`)
          .style("visibility", "inherit");
      
      const dateFormatted = moment.unix(parseInt(ym)/1e9).format("MMM Do");
      line.select(".text-year")
        .text(dateFormatted);
      
      line.selectAll(".text-rating")
        .text(d => {
          return Math.floor(parseInt(d.time)/(24*60*60*1e9)) === Math.floor(parseInt(ym)/(24*60*60*1e9)) ? (d[genre])+" page loads/ 1k students (log10)" : "";
        });

      line.selectAll(".text-genre")
        .style("visibility", "hidden")
        .filter(d => d.key === genre)
        .style("visibility", "inherit")
        .attr("fill", d => d.key === genre ? color(d.key) : "#000")
        .text(genre);
      
      line.selectAll(".cursor-line")
        .attr("stroke", "fff0")
        .filter(d => d.key === genre)
        .attr("stroke", d => d.key === genre ? color(d.key) : "#fff0")
      
      path.attr("opacity", d => d.key === genre ? 0.8 : 0.3);
      
      if (genre === null) {
        path.attr("opacity", 0.8);
        line.style("visibility", "hidden");
      }
    }

    const mouseenter = () => {
      line.attr("display", null);
    }
  
    const mouseleave = () => {
      line.attr("display", "none");
    }

    svg
        .on("mousemove", mousemove)
        .on("mouseenter", mouseenter)
        .on("mouseleave", mouseleave)

  }

  svg.call(hover, path);

  // Pandemic Start
  svg.append("text")
  .attr("x", "42%")
  .attr("y", "21%")
  .attr("font-size", "20px")
  .attr("text-anchor", "middle")
  .attr('fill', "black")
  .attr("font-weight", 900)
  .style("pointer-events", "none")
  .text("Pandemic Start");

});

});
'''

h = display(HTML(htmlt7))
j = py_display.Javascript(js_t7)
py_display.display_javascript(j)

<div class="alert simple-alert" style="font-family: times-new-roman">
  <p style="font-family: times-new-roman"><b>📌 Note</b>: So, the fact that the engagement didn't change as much between before and after the start of the pandemic for the top most used products shows that there are pupils that either don't have access or don't want to engage as much with the e-tools and the new scheduled school life in general. We could also ask the teachers whether or not this statement is true, because it depends on the fact that they weren't using much of the e-learning tools before the pandemic, but made use on more traditional learning products like books, manuals, movies and in-class games and activities.</p>
</div>

<h3 style="font-family: times-new-roman"> 🏫 Learning Product Engagement on States </h3>

<ul style="font-family: times-new-roman">
  <li>Illinois, Indiana, New York, Connecticut and Ohio are the states with the biggest engagement in almost all 10 learning products.</li>
  <li>Utah, California, Florida, Tennessee, Texas, Michigan, Minnesota and Virginia are the states with the lowest engagement in almost all 10 learning products.</li>
</ul>

In [ ]:
# Get mean info on product id and district
prodwheel = df.groupby(["lp_id", "district_id"])["engagement_index"].mean().\
                reset_index()
prodwheel = pd.merge(prodwheel, prods[["LP ID", "Product Name"]], 
                     left_on="lp_id", right_on="LP ID")
prodwheel = prodwheel[prodwheel["lp_id"].isin(top_n)]

# Add state info
prodwheel = pd.pivot(data=prodwheel, index="district_id", 
                     columns="Product Name", values="engagement_index").\
                    reset_index()
prod_names = list(prodwheel.columns[1:])
prodwheel = pd.merge(prodwheel, districts[["district_id", "state", 
                                           "Abbreviation", "Region"]],
                     on="district_id")
prodwheel = prodwheel.groupby(["state","Abbreviation", "Region"])[prod_names].\
                mean().reset_index()
prodwheel.rename(columns={'state': 'State'}, inplace=True)

# Convert the data to Maximum/Partial/Unclear
for col in prod_names:
    thresh = prodwheel[col].median()
    prodwheel[col] = np.where(pd.isnull(prodwheel[col]) == True,
                         'Unclear', np.where(prodwheel[col] >= thresh, 
                                             'Maximum', 'Partial'))
    
prodwheel.to_csv("prodwheel.csv", index=False)


htmlt6 = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.andradutza {
		font-family: "Times New Roman", Times, serif;
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.slider {
	-webkit-appearance: none; 
	appearance: none;
	width: 10%; 
	height: 15px; 
	background: #e7e7e7; 
	outline: none;
	opacity: 0.7; 
	border-radius: 6px;
	-webkit-transition: .2s; 
	transition: opacity .2s;
	}

	/* Mouse-over effects */
	.slider:hover {
	opacity: 1;
	}

	.slider::-webkit-slider-thumb {
	-webkit-appearance: none; /* Override default look */
	appearance: none;
	width: 15px; /* Set a specific slider handle width */
	height: 15px; /* Slider handle height */
	background: #1ACEE6; /* Green background */
	cursor: round; /* Cursor on hover */
	border-radius: 6px;
	}

	div.tooltip-donut {
		position: absolute;
		text-align: center;
		padding: .3rem;
		background: #FFFFFF;
		color: #000000;
		border: 1px solid #000000;
		border-radius: 8px;
		pointer-events: none;
		font-size: 0.8rem;
	}

</style>

<img id="baseimg" src="usa.png" style="display:none" />
<div class="andradutza">
	<center><h1>Yearly Mean Engagement</h1></center>
	<center><h1>on top 10 Learning Products</h1></center>
    <div class="row">
		<div class="rangeslider">
		</div>
		<div id="map5" class="svg-container"></div>
	</div>
	<div class="row">
		<input type="range" min="1" max="22" 
		value="10" class="slider" id="myRange"
		onchange="updateTextInput(this.value);">
	</div>
</div>
'''


js_t6 = '''
require.config({
  paths: {
    d3: "https://d3js.org/d3.v5.min",
    d3Array: "https://d3js.org/d3-array.v2.min",
  },
});

require(["d3", "d3Array"], function(d3, d3Array) {

// Inject d3array methods to d3
for (var key in d3Array) {
  d3[key] = d3Array[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/usa.png.*$/, '');

// Set the SVG area
const MARGIN = { LEFT: 100, RIGHT: 10, TOP: 10, BOTTOM: 100 }
const WIDTH = 950 - MARGIN.LEFT - MARGIN.RIGHT
const HEIGHT = 900 - MARGIN.TOP - MARGIN.BOTTOM

const svg = d3.select("#map5").append("svg")
  //.attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true);

const g = svg.append("g")
  .attr("transform", `translate(${MARGIN.LEFT}, ${MARGIN.TOP})`)


// === Wheel ===
d3.csv(baseAssetsUrl + "prodwheel.csv").then(function(data){

  // Wheel Rotation
  var rotate = document.getElementById("myRange").value;

  // Base Data
  const regionOrder = ["Northeast", "Midwest", "West", "South"]
  var wideData = data.sort(
    (a, b) =>
      d3.ascending(
        regionOrder.indexOf(a.Region),
        regionOrder.indexOf(b.Region)
      ) || d3.ascending(a.Abbreviation, b.Abbreviation)
  )

  // === Constants ===
  var rotation = (rotate / wideData.length) * Math.PI * 2
  var categories = wideData.columns.slice(3)
  var categoryOrder = categories.slice().reverse()
  const innerRadius = 155
  const outerRadius = WIDTH / 2
  const colors = ["#db222a", "#e16853", "#f7b227", "#b4c95a",
                "#1be9df", "#1acee6", "#5d6dba", "#5c4d86",
                "#af6eac", "#c1385c"]

  var grouped = d3.groups(wideData, d => d.Region)
  var color = d3
    .scaleOrdinal()
    .domain(categories)
    .range(colors)
    .unknown("#505050")

  // Scales
  const x = d3
    .scaleBand()
    .domain(wideData.map(d => d.State))
    .range([rotation, 2 * Math.PI + rotation])

  const y = d3.scaleLinear()
    .domain([0, categories.length])
    .range([innerRadius, outerRadius])

  // Regions Data
  var regions = d3
    .groups(wideData, d => d.Region)
    .map(([region, states]) => {
      const startAngle = x(states[0].State);
      const endAngle = 
          x(states[states.length - 1].State) + x.bandwidth();
      const labelAngle =
        ((startAngle + endAngle) / 2 - Math.PI / 2) % (Math.PI * 2);
      return {
        region,
        startAngle,
        endAngle,
        labelAngle
      };
    })

  // Long Data (for main chart)
  var longData = wideData.flatMap(row =>
    categories.map(category => ({
      state: row.State,
      abbreviation: row.Abbreviation,
      region: row.Region,
      category,
      status: row[category]
    }))
  )

  // ARC
  var arc = d3
  .arc()
  .innerRadius(d => y(categoryOrder.indexOf(d.category)))
  .outerRadius(d => y(categoryOrder.indexOf(d.category) + 1))
  .startAngle(d => x(d.state))
  .endAngle(d => x(d.state) + x.bandwidth())
  .padRadius(innerRadius)

  // Label X and Y
  var labelX = (angle, radius) => Math.cos(angle) * radius
  var labelY = (angle, radius) => Math.sin(angle) * radius

  // Banned X and Y
  var bannedX = d =>
    labelX(
      x(d.state) + x.bandwidth() / 2 - Math.PI / 2,
      y(categoryOrder.indexOf(d.category) + 0.5)
    )
  var bannedY = d =>
    labelY(
      x(d.state) + x.bandwidth() / 2 - Math.PI / 2,
      y(categoryOrder.indexOf(d.category) + 0.5)
    )


  // ====================================
  // =============== CHART ==============
  // ====================================
  const wheel = g.selectAll()
  .data(longData)
  .join("g");

  wheel.append("path")
    .attr("d", arc)
    .attr("stroke", "white")
    .attr("fill", d =>
      color(["Maximum", "Partial"]
      .includes(d.status) ? d.category : null))
    .attr("fill-opacity", 
      d => (d.status === "Partial" ? 0.4 : 1))
    .attr('transform', 
    'translate('+(WIDTH/2.2) +','+(HEIGHT/1.75)+')');

  wheel
    .append("title")
    .text(d => `${d.state}: ${d.category}, ${d.status}`);

  // Region Labels
  var labelArc = d3
  .arc()
  .innerRadius(innerRadius - 20)
  .outerRadius(innerRadius - 20)
  .startAngle(d => d.startAngle)
  .endAngle(d => d.endAngle)

  // State Labels
  wheel
    .append("g")
    .attr("text-anchor", "middle")
    .attr("fill", "#888")
    .selectAll("text")
    .data(wideData)
    .join("text")
    .attr("x", d =>
      labelX(x(d.State) + x.bandwidth() / 2 - Math.PI / 2, outerRadius + 16)
    )
    .attr("y", d =>
      labelY(x(d.State) + x.bandwidth() / 2 - Math.PI / 2, outerRadius + 16)
    )
    .attr("dy", "0.31em")
    .text(d => d.Abbreviation)
    .attr('transform', 
    'translate('+(WIDTH/2.2) +','+(HEIGHT/1.75)+')');

  // LEGEND
  const legend = g
    .selectAll("wheel")
    .data(categories.concat(["Unclear"]))
    .join("g")
    .attr(
      "transform",
      (d, i) => `translate(325,
          ${(i - ((categories.length + 1)/5)+28) * 20})`
    );

  legend
    .filter(d => categories.includes(d))
    .append("rect")
    .attr("x", -20)
    .attr("y", -160)
    .attr("width", 36)
    .attr("height", 18)
    .style("opacity", 0.4)
    .attr("fill", color);

  legend
    .append("rect")
    .attr("x", 30)
    .attr("y", -160)
    .attr("width", 30)
    .attr("height", 18)
    .attr("fill", color);

  legend
    .append("text")
    .attr("x", 65)
    .attr("y", -150)
    .attr("dy", "0.35em")
    .attr("font-size", "12px")
    .text(d => d);

  g.append("text")
    .attr("y", "38.5%")
    .attr("x", "33.5%")
    .attr("font-size", "12px")
    .attr("text-anchor", "middle")
    .attr("font-weight", 900)
    .text("<Median")

  g.append("text")
    .attr("y", "38.5%")
    .attr("x", "40%")
    .attr("font-size", "12px")
    .attr("text-anchor", "middle")
    .attr("font-weight", 900)
    .text(">Median")
});

});
'''

h = display(HTML(htmlt6))
j = py_display.Javascript(js_t6)
py_display.display_javascript(j)

<h1 style="font-family: times-new-roman">3. 🧠 State of Mental Health</h1>

<p style="font-family: times-new-roman">So far we have analyzed the data and took some personal conclusions to link certain patterns within the data with "feelings" or "beliefs" that we might have.<p>
    
<p style="font-family: times-new-roman">However, these beliefs are biased and reflect my own take on these results. Hence, I believe it's best to ask the teachers, the people that have been in the front line during all this time how to interpret the data and maybe what actions could we further take to make it all better in the future.<p>
    
<p style="font-family: times-new-roman">The analysis below is taken from <a href="https://e4e.org/voices-virtual-classroom">Voices from the Virtual Classroom</a> and it shows the key highlights and responses from teachers, what they have struggled with and what they've seen change in the behavior of their students.<p>
    

<h1 style="font-family: times-new-roman"> Getting deeper: </h1>

<ul style="font-family: times-new-roman">
  <li>All teachers agree that the apparition of <b>Covid-19 caused a big disruption in the academic life</b>. All use e-learning tools, but only ~half said that they were acquainted with it before. Looking in the original report we can observe that the <b>Primary teachers had the most "learning" to do in getting used to the educational e-learning tools</b>.</li>
  <li>Most teachers agree that <b>Homework/Assignments completion is worse than before</b>. Regarding <b>time spent on grading</b>, <b>Primary and Middle teachers say they spend less time</b> on it (because the children are very young and is very hard to correctly asses them), while the <b>High School teachers say they spend more time</b> (due to the fact that is much harder to evaluate online papers vs physical ones).</li>
  <li>A good outcome has been that teachers started to <b>develop more proffesionally</b>, as well as <b>reach out to parents and children more often</b> than in the past, in an effort to keep up their engagement and support as much as possible.</li>
  <li><b>Only ~50% of teachers state that their students have age appropriate skills</b> to handle the tech part of studying from home - the other half are mainly Primary and Middle class teachers, with children too young to fully understand how to navigate between the apps and acquire new information in the same time.</li>
</ul>


<div class="alert simple-alert" style="font-family: times-new-roman">
  <p style="font-family: times-new-roman"><b>📌 Serious issues</b>: Most teachers agree that the highest issues are the <b>little to no access to technological tools or skills</b>. Most parents can't afford buying a laptop for every child in the house, or help them whenever they enquire a technical issue. There is also an issue with the <b>lack of space</b>, as the children and parents usually study/work in the same room. <b>Broadband is another problem</b>, as online classes happen in the same time in different regions, so the members of the call may get disconnected at any time. Lastly, the main concern for teaches is the <b>capacity to "get back on track" with the curriculum</b>, while students are more anxious about the <b>Social and Emotional implications</b> Covid-19 might have on their academic life.</p>
</div>

In [ ]:
# Import and save dataset
teachers = pd.read_csv("../input/covid19-education-data/Teacher Survey 2020.csv")

# # Save into W&B
save_dataset_artifact(run_name="teachers_survey", artifact_name="teachers_survey",
                      path="../input/covid19-education-data/Teacher Survey 2020.csv")
# Drop same questions
teachers = teachers.drop([2, 7, 8, 13, 18]).reset_index(drop=True)
# Unicize for D3 the Answer column
tag = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, "a",
       "b", "c", "d", "e", "f", "g", "h"]
teachers["Answer"] = teachers["Answer"] + tag

teachers.to_csv("teachers_survey.csv", index=False)


htmlt8 = '''
<!-- Bootstrap -->
<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
<!-- Style -->
<style>
	.andradutza {
		font-family: "Times New Roman", Times, serif;
	}
	.svg-container {
		display: inline-block;
		position: relative;
		width: 100%;
		padding-bottom: 100%;
		vertical-align: top;
		overflow: hidden;
	}
	.svg-content {
		display: inline-block;
		position: absolute;
		top: 0;
		left: 0;
	}

	.axisRed path{
	stroke: #474747;
	}

	#map8 .annotation text {
		fill: #FF0F7B;
	}

</style>

<img id="baseimg" src="usa.png" style="display:none" />
<div class="andradutza" style="background-color:#474747;">
	<center><h1 style="color:white;">🧠 The State of Mental Health</h1></center>
	<center><h4 style="color:white;">-Key Highlights & Takeaways-</h4></center>
	<div class="row">
		<div id="map8" class="svg-container"></div>
	</div>
</div>
'''


js_t8 = '''
require.config({
  paths: {
    d3src: "https://d3js.org/",
  },
  map: {
    '*': {
      'd3v6': 'd3src/d3.v6.min',
      'd3-selection': 'd3src/d3-selection.v1.min',
      'd3-drag': 'd3src/d3-drag.v1.min',
      'd3-shape': 'd3src/d3-shape.v1.min',
      'd3-path': 'd3src/d3-path.v1.min',
      'd3-dispatch': 'd3src/d3-dispatch.v1.min',
      'd3-annotation': 'https://cdnjs.cloudflare.com/ajax/libs/d3-annotation/2.5.1/d3-annotation.min.js',
    }
  }
});

require(["d3v6", "d3-annotation"], function(d3, d3Annotation) {

  function wrap(text, width) {
    text.each(function() {
      var text = d3.select(this),
          words = text.text().split(/\s+/).reverse(),
          word,
          line = [],
          lineNumber = 0,
          lineHeight = 1.1, // ems
          y = text.attr("y"),
          dy = parseFloat(text.attr("dy")),
          tspan = text.text(null).append("tspan").attr("x", 0).attr("y", y).attr("dy", dy + "em")
      while (word = words.pop()) {
        line.push(word)
        tspan.text(line.join(" "))
        if (tspan.node().getComputedTextLength() > width) {
          line.pop()
          tspan.text(line.join(" "))
          line = [word]
          tspan = text.append("tspan").attr("x", 0).attr("y", y).attr("dy", `${++lineNumber * lineHeight + dy}em`).text(word)
        }
      }
    })
  } 

// Inject d3Annotation methods to d3
for (var key in d3Annotation) {
  d3[key] = d3Annotation[key];
}

// Get Assets URL set by Kaggle
const baseAssetsUrl = document.getElementById('baseimg').src.replace(/usa.png.*$/, '');

// Set the SVG area
const margin = { LEFT: 30, RIGHT: 30, TOP: 10, BOTTOM: 30 }
const width = 950 - margin.LEFT - margin.RIGHT
const height = 900 - margin.TOP - margin.BOTTOM

const svg = d3.select("#map8").append("svg")
  // .attr("style", "outline: thin solid red;")
  .attr("preserveAspectRatio", "xMinYMin meet")
  .attr("viewBox", "0 0 950 900")
  .classed("svg-content", true)
  .attr("transform",
   `translate(${margin.LEFT}, ${margin.TOP})`)


// === Bubble Chart ===
d3.csv(baseAssetsUrl + "teachers_survey.csv").then(function(data){ 

  data.forEach(d => {
    d.Percentage = Number(d.Percentage)
  })

  // Scales
  const x = d3.scaleLinear()
    .range([0, height/5])
    .domain([0, d3.max(data, d => d.Percentage)])

  const y = d3.scalePoint()
    .range([0, width/1.02])
    .domain(data.map(d => d.Question))
    .padding(0.4)

  const y_2 = d3.scalePoint()
  .range([0, width/1.02])
  .domain(data.map(d => d.Answer))
  .padding(0.4)

  // Scale positions
  const xAxisGroup = svg.append("g")
  .attr("class", "axisRed")
    .attr("transform", `translate(400, ${width/1})`)

  const yAxisGroup = svg.append("g")
  .attr("class", "axisRed")
    .attr("transform", `translate(400, ${width/5-160})`)

  const yAxisGroup_2 = svg.append("g")
  .attr("class", "axisRed")
  .attr("transform", `translate(500, ${width/5-160})`)
  
  
  // Axis Creation
  xAxisGroup.call(d3.axisBottom(x)
                    .tickSize(0)
                    .tickValues([]))
    .selectAll("text")

  yAxisGroup.call(d3.axisLeft(y)
                    .tickSize(0))
    .selectAll("text")
    .call(wrap, y.bandwidth()+360)
    .attr("fill", "black")
    .attr("font-size", "11px")
    .attr("text-anchor", "end")
    .attr("fill", "white")
    // .attr("font-weight", 600)

  yAxisGroup_2.call(d3.axisRight(y_2)
                      .tickFormat(d => d.slice(0, -1))
                      .tickSize(0))
  .selectAll("text")
  .attr("fill", "black")
  .attr("font-size", "11px")
  .attr("text-anchor", "start")
  .attr("fill", "white")
  .attr("font-weight", 600)

  // Circles
  const circles = svg.selectAll()
    .data(data)
    .enter().append('g')
    .append("ellipse")
    .attr("class", "bar")
    .attr("transform", `translate(200, ${width/5-160})`)
    .attr("cx", 250)
    .attr("cy", d => y(d.Question))
    .attr("rx", d => x(d.Percentage)/6.6+15)
    .attr("ry", d => x(d.Percentage)/6.6)
    .style("fill", "#f89b29")


  svg.append('g')
  .selectAll('text')
  .data(data)
  .enter().append('text')
  .text(d => d.Percentage*100+"%")
  .attr('font-size', 11)
  .attr("font-weight", 600)
  .attr('dx', 442)
  .attr('dy', d => y(d.Question)+21)
  .style("fill", "#474747")
  .style("pointer-events", "none")

// Interactive
var div = d3.select("body").append("div")
.attr("class", "tooltip-donut")
.style("opacity", 0);

circles
.on('mouseenter', function (event, dt) {
// MOUSE ON
d3.selectAll('.value')
      .attr('opacity', 0)

d3.select(this)
    .transition()
    .duration(100)
    .attr('opacity', 0.7)
    .style("fill", "#FF0F7B")
    .style("stroke-width", "1px")
    .style("stroke", "#f89b29")

d3.select(this) 
  .append("text")
  .transition().duration(100)
  .attr("class", "text")
  .text("Hello")
  .attr("x", "50%")
  .attr("y", "50%")
  .attr("font-size", "25px")
  .attr("fill",  "white");

// Makes the new div appear
div.transition()
    .duration(50)
    .style("opacity", 1);

})
// MOUSE LEAVE
.on('mouseleave', function () {
  d3.selectAll('.value')
      .attr('opacity', 1)

  d3.select(this)
  .transition()
  .duration(100)
  .attr('opacity', 1)
  .style("fill", "#f89b29")
})
  
});

// Note
const annot1 = [
  {
  note: { 
    label: "The teachers are split in half: 53% with lots of experience with e-learning, while 43% had no encounter with it.",
    title: "A Great Deal of Experience: 53%",
    wrap: 230,  // smaller text splits in multiple lines
    padding: 10,   // More = text lower
  },
  connector: {
    end: "dot"},
  color: ["#ffffff"],
  x: 643,
  y: 132,
  dy: -20,
  dx: 20
},
{
  note: { 
    label: "The responses are split evenly; this shows that children were affected differently according to their age and grade.",
    title: "Same: 33% | Less: 31%",
    wrap: 230,  // smaller text splits in multiple lines
    padding: 10,   // More = text lower
  },
  connector: {
    end: "dot"},
  color: ["#ffffff"],
  x: 605,
  y: 280,
  dy: -20,
  dx: 30
},
{
  note: { 
    label: "The majority are spending more on emotional support, however 31% are spending less time than before - showing an estrangement in communication from some of the children.",
    title: "Less than Before: 31%",
    wrap: 230,  // smaller text splits in multiple lines
    padding: 10,   // More = text lower
  },
  connector: {
    end: "dot"},
  color: ["#ffffff"],
  x: 603,
  y: 475,
  dy: -20,
  dx: 30
},
{
  note: { 
    label: "Higher grades are more skilled with technology, while little ones tend to struggle more with it (an need supervision).",
    title: "Most: 46%",
    wrap: 230,  // smaller text splits in multiple lines
    padding: 10,   // More = text lower
  },
  connector: {
    end: "dot"},
  color: ["#ffffff"],
  x: 598,
  y: 722,
  dy: -25,
  dx: 60
}]

svg.append("g")
  .style('font-size', 14)
  .call(d3.annotation()
          .annotations(annot1))
});
'''

h = display(HTML(htmlt8))
j = py_display.Javascript(js_t8)
py_display.display_javascript(j)

<h1 style="font-family: times-new-roman">📑 4. Taking all to action</h1>

<h2 style="font-family: times-new-roman">📑 What do we now know?</h2>

<center><img src="https://i.imgur.com/PpENQpl.png" width=800></center>

<ul style="font-family: times-new-roman">
  <li>Activity and Engagement increased a bit after the start of the pandemic and then slightly decreased in the beginning of the academic year 2020-2021 (however, this trend isn't shown in top 10 products). This means that both <b>teachers and students started using more diligently a few products that quickly monopolized the others</b>, however the overall average engagement is low and alarming, at least compared to what I would have expected to see.</li>
  <li>We learned that the states where there are bigger percentages of <b>black/hispanic students</b> or more <b>pupils that benefit of free/reduced lunch need better care</b> and attention. <b>North Dakota, Arizona, New York, New Hempshire and Connecticut</b> are the states that stand out in terms of the biggest engagement in comparison to the others. However, the overall low engagement may be caused by:</li>
 <ul style="font-family: times-new-roman">
    <li><b>Lack of age appropiate skills</b>: children in Primary and Middle school don't know yet how to properly use the technology and might get stuck and frustrated on the way. If the parent or tutor doesn't know, can't or isn't able to be there for them when they need it, they might fall short and stop using the educational products alltogether.</li>
    <li><b>Lack of technology resources</b>: families that have 2 or more children would need to use a distinct laptop/tablet in the same time during the online courses AND have a proper sepparate (quiet) space to learn. Unfortunately, many families lack these kind of resources.</li>
    <li><b>Low Emotional Stability</b>: truth is some children started getting tired of the "new norm". Despite the emotional support they have from their teacher, lack of interaction might have accumulated, leading to them not wanting to engage and participate during the online classes and/or during homework.</li>
    <li><b>Low energy</b>: Staying in from of a computer 6 hours a day and then doing the homework (also digitaly) for another 2 to 3 hours can take a toll on children's overall wellbeing. This accumulates to 8+ hours of staying in a chair, 5 times a week, equivalent to a full time job.</li>
  </ul>
  <li>There have been active efforts from teachers to <b>improve their technological skills as well as overall curriculum</b>, in an effort to adapt it as much as possible for the "online" classrooms. Moreover, they are active in keeping in touch and <b>reaching out more to parents and children</b>. These acts are extremely benefficial, narrowing the communication gap that has been inevitably formed while staying at home.</li>
</ul>

<h2 style="font-family: times-new-roman">📑 What to do better?</h2>

<center><img src="https://i.imgur.com/JqGi3AG.png" width=800></center>

<ol style="font-family: times-new-roman">
  <li><b>Keeping schools open as much as possible should be a priority</b>, as this solves many problems that the pandemic brought to begin with. To keep the safety, I would move into online format for 2 weeks only classes that have a confirmed case, ~10 days if no new case has been found between the other students.</li>
  <li>For online classes, <b>teachers should try and brainstorm as many new, fun, out of the ordinary lectures to keep the students engaged</b>. This way, they will join every day with a curiosity in mind: "What will we do today?". New games, movies or fun interactive ways to teach might encourage them to pay attention, despite being alone at home and not in a classroom full of people.</li>
  <li><b>Better marketing for e-learning products</b>. As we saw, there are only 6-7 products that ar highly used. This is because teachers do not know about other apps and do not have the necessary time to research. Hence, I would <b>designate a new job</b> (part time or full time), where the requirements would be to <b>always be up to date with e-learning tools and to recommend and explain to teachers what and how to use them</b>. Moreover, I would make this <b>customized</b> on a grade and lecture basis: a 1st grade teacher will need different e-learning tools vs. what a math teacher in 8th grade might need. Teachers already have their hands full, so a designated person that will share customized tools for everybody will help grow both the engagement of the students (bringing something new and exciting to the table) and the engagement of the product.</li>
  <li>The <b>e-learning tools should always be <i>FREE</i></b> for the end-user: either free by default or paid by the educational institute. Assuming that the teacher or parent can afford any additional subscription for e e-learning tool would be faulty and unrealistic.</li>
  <li>We should not underestimate the power of these e-learning tools. Even after the pandemic is over, teachers could still use these amazing well thought apps to show young future adults how to navigate through technology and to learn in an interactive different way. Helping the low engagement products gain visibility will benefit the pupils, teachers and overall market tremendously.</li>
  <li>Teachers should take part in <b>IT courses</b>, so they could get better acquinted with the digital world.</li>
  <li><b>Online classes should be shorter</b> and have different curriculum (and engaging, as stated at 2.). Teachers should also coordinate their assignments every week, so pupils would not have an overload of work at once. <b>Parents should be involved</b> as well, to be sure that not only children take part in all their online classes, but that they also have some time appart from the devices. <i>It would be a shame to shorten the time spent at the computer for online classes, but the child to immediately start a game or browse social media afterwards for hours on end</i>. <b>Encouraging them to get outside</b> in a safe manner will <b>benefit their Social and Emotional anxiety</b>.</li>
  <li>Introduce <b>more psychologists</b> in schools. I would also implement a mandatory 1to1 for each student and psychologist at least every few weeks. This way we can check in as adults and catch early any metal health issue or distress that the child might develop.</li>
  <li>A big issue I struggle to find a fix to is for smaller children. School's main purpose for Primary pupils is to involve and integrate them into the society, hence any online classes will struggle to meet this goal. Furthermore, little children will need mandatory help and guidance from their parents/tutors using the technology. Hence, I would encourage smaller sizes of classes and a lighter policy whenever any Covid-19 case is found, as their tolerance to the virus is very good.</li>
</ol>

<h2 style="font-family: times-new-roman">🌐 Just a thought ...</h2>

<center><img src="https://i.imgur.com/kPSYX20.jpg" width=800></center>

<p style="font-family: times-new-roman">What better time than this to start developing and implementing true <b>VR</b> in schools?</p>

<p style="font-family: times-new-roman">Although expensive and ... hey, listen! I am just daring to think of what it could be (thinking that pupils in US and all over the world lack at the moment the "simple" resources like a good reliable device, connection to internet or space to study), VR could take teaching to the next level.</p>

<p style="font-family: times-new-roman">I am inevitably thinking of these SUPER COOL holograms in Star Wars and their "online" meetings: all standing in their chair, but being able to communicate and see eachother in a real space and time, no matter their location in the Universe - Augmented Reality.</p>

<p style="font-family: times-new-roman">Taking this a step further and we could learn in a classroom in the Amazonian Jungle, in Space or on top of a building, being able to interact virtualy with the teacher and the classmates, all from our home - Virtual Reality.</p>

<p style="font-family: times-new-roman">And it has everything: the interactivity, the engagement, the social connection and the safety of the home.</p>

<p style="font-family: times-new-roman">I know it's a long shot. I know all the barriers. But ... wouldn't it be awesome? 🤯</p>

<center><img src="https://i.imgur.com/nF12w9Z.png" width=800></center>

<h1 style="font-family: times-new-roman">🙏 Giving thanks</h1>

<p style="font-family: times-new-roman">This analysis couldn't have been possible without the guidance and contribution of some amazing people and articles:</p>

<ul style="font-family: times-new-roman">
  <li><b>Schubert</b>: He and <a href="https://www.kaggle.com/spitfire2nd/enthusiast-to-data-professional-what-changes/">his second place winning notebook for the DS Kaggle Survey Challenge</a> showed me for the first time ever the beauty and wonders of D3. He also guided me onto how to start learning, and for his kindness I am forever grateful. </li>
  <li><b>D3 help</b>: How to create a mouse-over pie chart: <a href="https://medium.com/@kj_schmidt/show-data-on-mouse-over-with-d3-js-3bf598ff8fc2">by KJ Schmidt</a> and beautiful bar chart <a href="https://jsfiddle.net/matehu/w7h81xz2/">by matehu</a>.</li>
  <li><b>The beautiful wheel</b>: The wheel is customed after this amazing article here: <a href="https://observablehq.com/@kerryrodden/equal-area-radial-matrix-of-lgbt-rights">Equal-Area Radial Matrix of LGBT Rights</a>.</li>
  <li><b>Loris Mat</b>: For this beautiful <a href="https://observablehq.com/@git1984/streamgraph-pitchfork-reviews">streamgraph</a>.</li>
</ul>

<hr size="4" width="100%" color="#5d6dba">

> <p style="font-family: times-new-roman">My <b><a href="https://wandb.ai/andrada/C19-learn?workspace=user-andrada">W&B Dashboard</a></b> is growing:<p>

<center><img src="https://i.imgur.com/RFKeYrv.png" width=1000></center>
<br>
<img src="https://i.imgur.com/cUQXtS7.png">
<h1 style="font-family: times-new-roman">My Specs</h1>

<ul style="font-family: times-new-roman">
  <li><b>🖥 Z8 G4</b>: Workstation</li>
  <li>💾 2 CPUs & 96GB Memory</li>
  <li><b>🎮 NVIDIA</b> Quadro RTX 8000</li>
  <li><b>💻 Zbook</b> Studio G7 on the go</li>
</ul>